In [22]:
import os
import pandas as pd
import csv
import re
import numpy as np
import datetime
import pytz
from google.cloud import bigquery
from google.cloud.bigquery import SchemaField


## Process Roster CSV file

In [265]:

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Roster/')

file="meeting-26886-results.csv"


df = pd.read_csv(file, dtype=str)


In [266]:
df.describe()

,MeetingId,MeetingName,MeetingDate,VenueName,Organiser,City,Country,Season,EntryId,StartListId,...,Round24Attempts,Round24TargetHeight,Round24Wind,Round25Status,Round25Record,Round25Records,Round25Result,Round25Attempts,Round25TargetHeight,Round25Wind
count,4320,4320,4320,4320,4320,4320,4320,4320,4320,4320,...,1,6,0,6,0,0,0,1,6,0
unique,1,1,1,1,1,1,1,1,4320,217,...,1,1,0,1,0,0,0,1,1,0
top,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2451719,313732,...,xo,4.75,NaN,ok,NaN,NaN,NaN,xxx,4.85,NaN
freq,4320,4320,4320,4320,4320,4320,4320,4320,1,124,...,1,6,NaN,6,NaN,NaN,NaN,1,6,NaN


In [267]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4320 entries, 0 to 4319
Data columns (total 247 columns):
 #    Column                   Non-Null Count  Dtype 
---   ------                   --------------  ----- 
 0    MeetingId                4320 non-null   object
 1    MeetingName              4320 non-null   object
 2    MeetingDate              4320 non-null   object
 3    VenueName                4320 non-null   object
 4    Organiser                4320 non-null   object
 5    City                     4320 non-null   object
 6    Country                  4320 non-null   object
 7    Season                   4320 non-null   object
 8    EntryId                  4320 non-null   object
 9    StartListId              4320 non-null   object
 10   Title                    814 non-null    object
 11   RelayTeamName            238 non-null    object
 12   NationalRelayTeam        238 non-null    object
 13   FullName                 4082 non-null   object
 14   FirstName             

In [268]:
df

,MeetingId,MeetingName,MeetingDate,VenueName,Organiser,City,Country,Season,EntryId,StartListId,...,Round24Attempts,Round24TargetHeight,Round24Wind,Round25Status,Round25Record,Round25Records,Round25Result,Round25Attempts,Round25TargetHeight,Round25Wind
0,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2451719,313711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2451722,313711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2451714,313711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2451705,313711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2451717,313711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2452695,313732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4316,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2452696,313732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4317,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2452648,313732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4318,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2452649,313732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [269]:
df.columns

Index(['MeetingId', 'MeetingName', 'MeetingDate', 'VenueName', 'Organiser',
       'City', 'Country', 'Season', 'EntryId', 'StartListId',
       ...
       'Round24Attempts', 'Round24TargetHeight', 'Round24Wind',
       'Round25Status', 'Round25Record', 'Round25Records', 'Round25Result',
       'Round25Attempts', 'Round25TargetHeight', 'Round25Wind'],
      dtype='object', length=247)

In [270]:
#for column in df.columns[72:]:
#    print(column)

In [271]:
# Use boolean masking to update full_eventnam field fo tris
# This is the only additional step for triathlon events


#df.loc[df['Event_name'] == 'Triathlon', 'Full_Eventname'] = 'Triathlon'+ ' - ' + df['Event_dist'] +' '+df['MultiSubEvent_name']
#df.loc[df['Event_name'] == 'Outdoor Pentathlon', 'Full_Eventname'] = 'Outdoor Pentathlon'+ ' - ' + df['Event_dist'] + ' '+ df['MultiSubEvent_name']
#df.loc[df['Event_name'] == 'Decathlon', 'Full_Eventname'] = 'Decathlon'+ ' - ' + df['Event_dist'] +' '+df['MultiSubEvent_name']
#df.loc[df['Event_name'] == 'Heptathlon', 'Full_Eventname'] = 'Heptathlon'+ ' - ' + df['Event_dist'] +' '+df['MultiSubEvent_name']


In [272]:
# Select relevant columns


df_filtered = df[['MeetingName', 'MeetingDate', 'VenueName', 'City', 'EntryId', 'RelayTeamName', 'NationalRelayTeam', 'FullName', 'FirstName','MiddleName', 'LastName','EventGender', 'ParaClassRunJump',
                 'ParaClassThrow', 'NotesPublic', 'CountryCode', 'DateOfBirth', 'EventStart', 'EventName','PrimaryImplementValue', 'PrimaryImplementUnit', 'EventStage', 'ClubName', 'BibNumber', 
                  'StartStatus', 'PersonalBest', 'SeasonBest', 'Lane', 'Place', 'AgeGroup', 'Title', 'WindReading', 'Promotion', 'HeatName', 'Result', 'ReactionTime','SeedingResult', 'ResultNET', 
                  'PersonalBest', 'SeasonBest', 'SplitTime', 'LapTime', 'RelayId']].copy()

df_attempts = df.iloc[:, 72:]

In [273]:
df_attempts

,Round1Status,Round1Record,Round1Records,Round1Result,Round1Attempts,Round1TargetHeight,Round1Wind,Round2Status,Round2Record,Round2Records,...,Round24Attempts,Round24TargetHeight,Round24Wind,Round25Status,Round25Record,Round25Records,Round25Result,Round25Attempts,Round25TargetHeight,Round25Wind
0,ok,SB,SB,13.656,NaN,NaN,-1.4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ok,NaN,NaN,13.716,NaN,NaN,-1.4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ok,PB,PB,13.960,NaN,NaN,-1.3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ok,PB,PB,14.119,NaN,NaN,-1.8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ok,PB,PB,14.132,NaN,NaN,-1.3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [282]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Roster/')

df_attempts.to_csv('attempts.csv', encoding='utf-8')


In [291]:
'''
# Does not exclude nan results
# Find all round numbers from the "Status" column names
round_numbers = sorted({
    int(re.search(r'Round(\d+)Status', col).group(1))
    for col in df_attempts.columns if re.match(r'Round\d+Status', col)
})

valid_rounds_list = []
for _, row in df_attempts.iterrows():
    rounds_dict = {}
    for rnd in round_numbers:
        status_col = f'Round{rnd}Status'
        result_col = f'Round{rnd}Result'
        wind_col = f'Round{rnd}Wind'

        if str(row[status_col]).strip().lower() == 'ok':
            rounds_dict[rnd] = {
                'Result': row[result_col],
                'Wind': row[wind_col]
            }
    valid_rounds_list.append(rounds_dict)

# Create new DataFrame with the original index and the ValidRounds column
results_df = pd.DataFrame({
    'ValidRounds': valid_rounds_list
})

'''

In [294]:
# Build a list of dicts for each original row
valid_rounds_list = []
for _, row in df_attempts.iterrows():
    rounds_dict = {}
    for rnd in round_numbers:
        status_col = f'Round{rnd}Status'
        result_col = f'Round{rnd}Result'
        wind_col = f'Round{rnd}Wind'

        result_val = row[result_col]

        # Only include if status is 'ok' AND result is not NaN
        if (
            str(row[status_col]).strip().lower() == 'ok'
            and pd.notna(result_val)
        ):
            rounds_dict[rnd] = {
                'Result': result_val,
                'Wind': row[wind_col]
            }
    valid_rounds_list.append(rounds_dict)

# Create new DataFrame
results_df = pd.DataFrame({
    'ValidRounds': valid_rounds_list
})


In [295]:
results_df

,ValidRounds
0,"{1: {'Result': '13.656', 'Wind': '-1.4'}}"
1,"{1: {'Result': '13.716', 'Wind': '-1.4'}}"
2,"{1: {'Result': '13.960', 'Wind': '-1.3'}}"
3,"{1: {'Result': '14.119', 'Wind': '-1.8'}}"
4,"{1: {'Result': '14.132', 'Wind': '-1.3'}}"
...,...
4315,{}
4316,{}
4317,{}
4318,{}


In [296]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Roster/')

results_df.to_csv('attempts_extracted.csv', encoding='utf-8')


In [274]:
# Potential new fields to map across:

# tilastopajaid
# relayid
# paraclass - EVENT_CLASS
# split time
# lap time
# resultnet



In [275]:
df_filtered.drop_duplicates(inplace=True)    


In [276]:
df_filtered

,MeetingName,MeetingDate,VenueName,City,EntryId,RelayTeamName,NationalRelayTeam,FullName,FirstName,MiddleName,...,HeatName,Result,ReactionTime,SeedingResult,ResultNET,PersonalBest,SeasonBest,SplitTime,LapTime,RelayId
0,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore,2451719,NaN,NaN,Jalea Yeo,Jalea,NaN,...,6,13.656,NaN,NaN,NaN,13.18,NaN,NaN,NaN,NaN
1,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore,2451722,NaN,NaN,Leia Ong,Leia,NaN,...,1,13.716,NaN,NaN,NaN,13.39,13.39,NaN,NaN,NaN
2,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore,2451714,NaN,NaN,Zi Xun Lui,Zi Xun,NaN,...,5,13.960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore,2451705,NaN,NaN,Kristy Kau,Kristy,NaN,...,4,14.119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore,2451717,NaN,NaN,Riana Kaul,Riana,NaN,...,5,14.132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore,2452695,NaN,NaN,Sentha Silva,Sentha,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2452694
4316,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore,2452696,NaN,NaN,Milan Ocean Ganesh,Milan Ocean,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2452694
4317,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore,2452648,Chun Bao Bao,false,NaN,NaN,NaN,...,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4318,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore,2452649,NaN,NaN,Yao Hui Chun,Yao Hui,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2452648


In [277]:
relay_ids = set(df_filtered['RelayId'].to_list())

In [278]:
len(relay_ids)

233

In [279]:
relay_names = set(df_filtered['RelayTeamName'].to_list())

In [280]:
len(relay_names)

202

In [281]:
# Separate out the relay data

relay_data = df_filtered[df_filtered['RelayTeamName'].notnull()|df_filtered['RelayId'].notnull()]

In [256]:
relay_data

,MeetingName,MeetingDate,VenueName,City,EntryId,RelayTeamName,NationalRelayTeam,FullName,FirstName,MiddleName,...,HeatName,Result,ReactionTime,SeedingResult,ResultNET,PersonalBest,SeasonBest,SplitTime,LapTime,RelayId
2600,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore,2451916,Team Cicada Trackers,false,NaN,NaN,NaN,...,0,52.665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2601,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore,2451917,NaN,NaN,Ching Lee,Ching,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2451916
2602,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore,2451918,NaN,NaN,Zurie Chong,Zurie,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2451916
2603,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore,2451919,NaN,NaN,Kristy Kau,Kristy,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2451916
2604,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore,2451920,NaN,NaN,Riana Kaul,Riana,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2451916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore,2452695,NaN,NaN,Sentha Silva,Sentha,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2452694
4316,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore,2452696,NaN,NaN,Milan Ocean Ganesh,Milan Ocean,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2452694
4317,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore,2452648,Chun Bao Bao,false,NaN,NaN,NaN,...,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4318,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore,2452649,NaN,NaN,Yao Hui Chun,Yao Hui,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2452648


In [257]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Roster/')

relay_data.to_csv('relay_data.csv', encoding='utf-8')


In [260]:
df=relay_data.copy()

In [261]:
# Forward-fill the team name so each member gets the right one
df["RelayTeamName"] = df["RelayTeamName"].replace("", pd.NA).ffill()

# Remove the pure team name row (where FullName is NaN or empty)
members_df = df[df["FullName"].notna() & (df["FullName"] != "")]

# Group by RelayId
relay_teams = (
    members_df.groupby("RelayId")
    .apply(lambda g: pd.Series({
        "RelayTeamName": g["RelayTeamName"].iloc[0],
        "Members": list(g["FullName"])
    }))
    .reset_index(drop=True)
)


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_23004/3877380155.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


In [ ]:
'''
Step 1: members_df.groupby("RelayId")
This tells pandas:

“Split my DataFrame into chunks (groups) where all rows in a chunk have the same RelayId.”

Example — from your CSV:

nginx
Copy
Edit
RelayId   FullName         RelayTeamName
2451916   Ching Lee        Team Cicada Trackers
2451916   Zurie Chong      Team Cicada Trackers
2451916   Kristy Kau       Team Cicada Trackers
2451916   Riana Kaul       Team Cicada Trackers
→ All these rows become one group because they have the same RelayId.

Step 2: .apply(lambda g: ...)
The .apply() method says:

“For each group g created by groupby, run this custom function and return the results.”

Here, g is a mini DataFrame containing just the rows for one RelayId.

Step 3: pd.Series({ ... }) inside apply
Inside the function, we:

Take the first team name from the group:
g["RelayTeamName"].iloc[0]
(Since all rows in a group have the same team name, we just grab the first.)

Turn all the names in FullName into a Python list:
list(g["FullName"])

We pack these into a pd.Series:

python
Copy
Edit
pd.Series({
    "RelayTeamName": g["RelayTeamName"].iloc[0],
    "Members": list(g["FullName"])
})
This is the summary row for that team.

Step 4: .reset_index(drop=True)
groupby.apply() returns a DataFrame where the group keys (RelayId) are part of the index.
We call .reset_index(drop=True) to:

Throw away the old index

Give the result a fresh 0..N index

✅ End result:
A clean DataFrame with one row per relay team, and:

RelayTeamName → team’s name

Members → list of names

'''

In [263]:
'''
1. Original Data
pgsql
Copy
Edit
RelayTeamName         FullName        RelayId
---------------------------------------------------
Team Cicada Trackers  Ching Lee       2451916
NaN                   Zurie Chong     2451916
NaN                   Kristy Kau      2451916
NaN                   Riana Kaul      2451916
Team Road Runners     John Tan        2487215
NaN                   Alex Wong       2487215
NaN                   Sarah Lim       2487215
2. Step 1: groupby("RelayId")
We now have two groups:

Group 1 — RelayId 2451916

pgsql
Copy
Edit
RelayTeamName         FullName        RelayId
---------------------------------------------------
Team Cicada Trackers  Ching Lee       2451916
NaN                   Zurie Chong     2451916
NaN                   Kristy Kau      2451916
NaN                   Riana Kaul      2451916
Group 2 — RelayId 2487215

markdown
Copy
Edit
RelayTeamName         FullName        RelayId
---------------------------------------------------
Team Road Runners     John Tan        2487215
NaN                   Alex Wong       2487215
NaN                   Sarah Lim       2487215
3. Step 2: .apply(lambda g: ...)
We now process each mini-DataFrame (g) separately.

For Group 1:

python
Copy
Edit
RelayTeamName = g["RelayTeamName"].iloc[0]   # "Team Cicada Trackers"
Members = list(g["FullName"])                # ["Ching Lee", "Zurie Chong", "Kristy Kau", "Riana Kaul"]
For Group 2:

python
Copy
Edit
RelayTeamName = g["RelayTeamName"].iloc[0]   # "Team Road Runners"
Members = list(g["FullName"])                # ["John Tan", "Alex Wong", "Sarah Lim"]
4. Step 3: Combine Results
After processing each group, pandas stacks the results into a new DataFrame:

markdown
Copy
Edit
RelayTeamName         Members
---------------------------------------------------
Team Cicada Trackers  [Ching Lee, Zurie Chong, Kristy Kau, Riana Kaul]
Team Road Runners     [John Tan, Alex Wong, Sarah Lim]
💡 Key point:
groupby → splits data into chunks → apply runs custom aggregation per chunk → results are recombined into one DataFrame.

'''

'\n1. Original Data\npgsql\nCopy\nEdit\nRelayTeamName         FullName        RelayId\n---------------------------------------------------\nTeam Cicada Trackers  Ching Lee       2451916\nNaN                   Zurie Chong     2451916\nNaN                   Kristy Kau      2451916\nNaN                   Riana Kaul      2451916\nTeam Road Runners     John Tan        2487215\nNaN                   Alex Wong       2487215\nNaN                   Sarah Lim       2487215\n2. Step 1: groupby("RelayId")\nWe now have two groups:\n\nGroup 1 — RelayId 2451916\n\npgsql\nCopy\nEdit\nRelayTeamName         FullName        RelayId\n---------------------------------------------------\nTeam Cicada Trackers  Ching Lee       2451916\nNaN                   Zurie Chong     2451916\nNaN                   Kristy Kau      2451916\nNaN                   Riana Kaul      2451916\nGroup 2 — RelayId 2487215\n\nmarkdown\nCopy\nEdit\nRelayTeamName         FullName        RelayId\n---------------------------------------

In [262]:
relay_teams

,RelayTeamName,Members
0,OMG. The weather is sooooo hot,"[Kian Boon Lim, Jiayi Lim]"
1,SeeeK Family,"[Kenny Wai Seng Leong, Elissa En Hui Leong]"
2,Lucas Cheng Team,"[Tze Wee John Cheng, Sheng Tze Lucas Cheng]"
3,Zeldy,"[Felix Choo, Zelda Choo]"
4,McNuggets Eating Club,"[Harry Loh, Rachel Loh]"
...,...,...
227,Aden Seow & Parent,"[Jason Au, Aden Kai qing Seow]"
228,Jaden Seow & Parent,"[Jason Au, Jaden Yuan Qing Seow]"
229,Hannah Wong & Parent,"[Charlene Mingyan Han, Hannah Danielle Wong]"
230,Black,"[Javier Loh, Aden Tan, Ye Tao Tong, Siang Yang..."


In [264]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Roster/')


df_attempts.to_csv('attempts.csv', sep=',', encoding='utf-8-sig', index=False)

In [289]:
#df_filtered.to_csv('TESTING.csv', sep=',', encoding='utf-8-sig', index=False)

In [290]:
#df_filtered['Name']= df_filtered['LAST_NAME'] + ', '+ df_filtered['FIRST_NAME']

df_filtered['Name']= df_filtered['Last_name'] + ', '+ df_filtered['First_name']

#df_filtered.drop(['Last_name', 'First_name'], axis=1, inplace=True)



In [291]:
df_filtered['Name']

0                           Nova, Emilia
1      Napisatul Munawaroh, Siti Nazirah
2          Bahtiar, Attina Nurkamila Int
3                             KIM, MINJI
4                          CHIU, YU-TING
                     ...                
482                         Tan, Bernice
483                      Goh, Shing Ling
484                                  ,  
487                                  ,  
490                                  ,  
Name: Name, Length: 456, dtype: object

In [292]:
df_filtered['Relay1_Leg_Name']= df_filtered['RelayLeg1_Last_name'] + ', '+ df_filtered['RelayLeg1_First_name']
df_filtered['Relay2_Leg_Name']= df_filtered['RelayLeg2_Last_name'] + ', '+ df_filtered['RelayLeg2_First_name']
df_filtered['Relay3_Leg_Name']= df_filtered['RelayLeg3_Last_name'] + ', '+ df_filtered['RelayLeg3_First_name']
df_filtered['Relay4_Leg_Name']= df_filtered['RelayLeg4_Last_name'] + ', '+ df_filtered['RelayLeg4_First_name']
df_filtered['Relay5_Leg_Name']= df_filtered['RelayLeg5_Last_name'] + ', '+ df_filtered['RelayLeg5_First_name']
df_filtered['Relay6_Leg_Name']= df_filtered['RelayLeg6_Last_name'] + ', '+ df_filtered['RelayLeg6_First_name']
df_filtered['Relay7_Leg_Name']= df_filtered['RelayLeg7_Last_name'] + ', '+ df_filtered['RelayLeg7_First_name']
df_filtered['Relay8_Leg_Name']= df_filtered['RelayLeg8_Last_name'] + ', '+ df_filtered['RelayLeg8_First_name']

df_filtered.drop(['RelayLeg1_Last_name', 'RelayLeg1_First_name', 'RelayLeg2_Last_name', 'RelayLeg2_First_name',
                 'RelayLeg3_Last_name', 'RelayLeg3_First_name', 'RelayLeg4_Last_name', 'RelayLeg4_First_name',
                 'RelayLeg5_Last_name', 'RelayLeg5_First_name', 'RelayLeg6_Last_name', 'RelayLeg6_First_name',
                 'RelayLeg7_Last_name', 'RelayLeg7_First_name', 'RelayLeg8_Last_name', 'RelayLeg8_First_name'], axis=1, inplace=True)

df_filtered.drop_duplicates(inplace=True)

In [293]:
df_filtered

,Last_name,First_name,Pref_name,Citizen_of,Event_gender,Birth_date,Ath_age,Comp_no,Team_name.1,Event_name,...,Meet_location,Name,Relay1_Leg_Name,Relay2_Leg_Name,Relay3_Leg_Name,Relay4_Leg_Name,Relay5_Leg_Name,Relay6_Leg_Name,Relay7_Leg_Name,Relay8_Leg_Name
0,Nova,Emilia,nan,INA,F,1995-08-20 00:00:00.000,30.0,37,DKI Java,Javelin Throw,...,Singapore Sportshub,"Nova, Emilia",",",",",",",",",",",",",",",","
1,Napisatul Munawaroh,Siti Nazirah,nan,INA,F,1997-03-28 00:00:00.000,28.0,36,DKI Java,Javelin Throw,...,Singapore Sportshub,"Napisatul Munawaroh, Siti Nazirah",",",",",",",",",",",",",",",","
2,Bahtiar,Attina Nurkamila Int,nan,INA,F,1998-11-18 00:00:00.000,27.0,83,Indonesia,Javelin Throw,...,Singapore Sportshub,"Bahtiar, Attina Nurkamila Int",",",",",",",",",",",",",",",","
3,KIM,MINJI,nan,KOR,F,2007-01-02 00:00:00.000,18.0,236,South Korea,Javelin Throw,...,Singapore Sportshub,"KIM, MINJI",",",",",",",",",",",",",",",","
4,CHIU,YU-TING,nan,TPE,F,2000-10-22 00:00:00.000,25.0,3,Chinese Taipei,Javelin Throw,...,Singapore Sportshub,"CHIU, YU-TING",",",",",",",",",",",",",",",","
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,Tan,Bernice,nan,SIN,F,2000-11-16 00:00:00.000,25.0,104,Lacticbuds,Run,...,Singapore Sportshub,"Tan, Bernice",",",",",",",",",",",",",",",","
483,Goh,Shing Ling,nan,SIN,F,1999-06-07 00:00:00.000,26.0,252,TeamFabian,Run,...,Singapore Sportshub,"Goh, Shing Ling",",",",",",",",",",",",",",",","
484,,,nan,nan,X,nan,nan,0,Club Zoom,Relay,...,Singapore Sportshub,",","Marcus, Seah","Irdina, Erza","Ong, Steven","Tan, Olivia",",",",",",","Wee, Zi Rui"
487,,,nan,nan,X,nan,nan,0,Erovra Club,Relay,...,Singapore Sportshub,",","CHANG, SZE XIAN LAWRENCE","PEI YUAN HOW, MARIE ADORABELLE","GAN SEONG HOO, VALEN","TAN, SUI XIN",",",",",",",","


In [294]:
df_filtered.reset_index(inplace = True, drop = True)  # reset index after dropping duplicates


In [295]:
df_filtered['Name']

0                           Nova, Emilia
1      Napisatul Munawaroh, Siti Nazirah
2          Bahtiar, Attina Nurkamila Int
3                             KIM, MINJI
4                          CHIU, YU-TING
                     ...                
451                         Tan, Bernice
452                      Goh, Shing Ling
453                                  ,  
454                                  ,  
455                                  ,  
Name: Name, Length: 456, dtype: object

In [296]:
# create dict for relay names

for i in range(len(df_filtered)):
    
    list_no=[]
    list_names=[]
    
    if 'Relay' in df_filtered.loc[i, 'Event_name']:
 #   if 'Relay' in df_filtered.loc[i, 'EVENT_NAME']:
 
                
        list_no.append(df_filtered.loc[i, 'RelayLeg1_Ath_no'])
        list_no.append(df_filtered.loc[i, 'RelayLeg2_Ath_no'])
        list_no.append(df_filtered.loc[i, 'RelayLeg3_Ath_no'])
        list_no.append(df_filtered.loc[i, 'RelayLeg4_Ath_no'])
        list_no.append(df_filtered.loc[i, 'RelayLeg5_Ath_no'])
        list_no.append(df_filtered.loc[i, 'RelayLeg6_Ath_no'])
        list_no.append(df_filtered.loc[i, 'RelayLeg7_Ath_no'])
        list_no.append(df_filtered.loc[i, 'RelayLeg8_Ath_no'])
        
        
        list_names.append(df_filtered.loc[i, 'Relay1_Leg_Name'])
        list_names.append(df_filtered.loc[i, 'Relay2_Leg_Name'])
        list_names.append(df_filtered.loc[i, 'Relay3_Leg_Name'])
        list_names.append(df_filtered.loc[i, 'Relay4_Leg_Name'])
        list_names.append(df_filtered.loc[i, 'Relay5_Leg_Name'])
        list_names.append(df_filtered.loc[i, 'Relay6_Leg_Name'])
        list_names.append(df_filtered.loc[i, 'Relay7_Leg_Name'])
        list_names.append(df_filtered.loc[i, 'Relay8_Leg_Name'])
        
        
        list_no = [i for i in list_no if i != 0]
        
        list_names = [i for i in list_names if i != ' ,  ']   
                
        res = {}
        for key in list_no:
            for value in list_names:
                res[key] = value
                list_names.remove(value)
                
                break
                
        print(res)
        print(df_filtered.loc[i, 'Name'])
        
        
        df_filtered.at[i, 'Name'] = res  # must use .at for dict and [res] because of adding Res_lane
        
     #   df.replace({"Courses": dict})
        
     #   df_filtered.loc[i, 'Name'] = df_filtered.loc[i, 'Name'].replace('', res)

        
        
        

        
                           
                



{'332': 'Tamil Selvam, Subaraghav Hari', '180': 'Aswin Kumar, Suyamprakasam', '333': 'Clayton, Caleb', '177': 'Ang, Jamie El-Red'}
 ,  
{'332': 'Tamil Selvam, Subaraghav Hari', '180': 'Aswin Kumar, Suyamprakasam', '333': 'Clayton, Caleb', '177': 'Ang, Jamie El-Red'}
 ,  
{'141': 'Goh, Audie', '148': 'Lim, Joshua', '155': 'Selven, Ajay', '149': 'Loy, Caleb'}
 ,  
{'141': 'Goh, Audie', '148': 'Lim, Joshua', '155': 'Selven, Ajay', '149': 'Loy, Caleb'}
 ,  
{'266': 'QUEK, JUN JIE CALVIN', '231': 'MUNCHERJI, ZUBIN', '243': 'LEE, REUBEN RAINER', '248': 'THANA RAJAN, THIRUBEN'}
 ,  
{'266': 'QUEK, JUN JIE CALVIN', '231': 'MUNCHERJI, ZUBIN', '243': 'LEE, REUBEN RAINER', '248': 'THANA RAJAN, THIRUBEN'}
 ,  
{'342': 'Xue Yee, Ngoh', '90': 'Irdina, Erza', '343': 'Lin Jie, Ng', '344': 'Tan, Olivia'}
 ,  
{'342': 'Xue Yee, Ngoh', '90': 'Irdina, Erza', '343': 'Lin Jie, Ng', '344': 'Tan, Olivia'}
 ,  
{'342': 'Xue Yee, Ngoh', '90': 'Irdina, Erza', '343': 'Lin Jie, Ng', '344': 'Tan, Olivia'}
 ,  
{'25

In [297]:
df_filtered.drop(['Relay_Ltr', 'RelayLeg1_Ath_no', 'RelayLeg2_Ath_no', 'RelayLeg3_Ath_no', 'RelayLeg4_Ath_no', 
                  'RelayLeg5_Ath_no', 'RelayLeg6_Ath_no', 'RelayLeg7_Ath_no', 'RelayLeg8_Ath_no', 'Relay1_Leg_Name',
                  'Relay2_Leg_Name', 'Relay3_Leg_Name', 'Relay4_Leg_Name', 'Relay5_Leg_Name', 'Relay6_Leg_Name',
                  'Relay7_Leg_Name', 'Relay8_Leg_Name'], axis=1, inplace=True)


In [298]:
df_filtered['Name']=df_filtered['Name'].astype(str) # convert dict to str so duplicates can be dropped


In [299]:
df_filtered

,Last_name,First_name,Pref_name,Citizen_of,Event_gender,Birth_date,Ath_age,Comp_no,Team_name.1,Event_name,...,Div_name,Meet_name,Meet_start,Meet_end,MultiSubEvent_name,Event_note,Event_dist,Event_score,Meet_location,Name
0,Nova,Emilia,nan,INA,F,1995-08-20 00:00:00.000,30.0,37,DKI Java,Javelin Throw,...,Open,85th Singapore Open Track & Field,2025-04-24 00:00:00.000,2025-04-25 00:00:00.000,,,0.0,0.0,Singapore Sportshub,"Nova, Emilia"
1,Napisatul Munawaroh,Siti Nazirah,nan,INA,F,1997-03-28 00:00:00.000,28.0,36,DKI Java,Javelin Throw,...,Open,85th Singapore Open Track & Field,2025-04-24 00:00:00.000,2025-04-25 00:00:00.000,,,0.0,0.0,Singapore Sportshub,"Napisatul Munawaroh, Siti Nazirah"
2,Bahtiar,Attina Nurkamila Int,nan,INA,F,1998-11-18 00:00:00.000,27.0,83,Indonesia,Javelin Throw,...,Open,85th Singapore Open Track & Field,2025-04-24 00:00:00.000,2025-04-25 00:00:00.000,,,0.0,0.0,Singapore Sportshub,"Bahtiar, Attina Nurkamila Int"
3,KIM,MINJI,nan,KOR,F,2007-01-02 00:00:00.000,18.0,236,South Korea,Javelin Throw,...,Open,85th Singapore Open Track & Field,2025-04-24 00:00:00.000,2025-04-25 00:00:00.000,,,0.0,0.0,Singapore Sportshub,"KIM, MINJI"
4,CHIU,YU-TING,nan,TPE,F,2000-10-22 00:00:00.000,25.0,3,Chinese Taipei,Javelin Throw,...,Open,85th Singapore Open Track & Field,2025-04-24 00:00:00.000,2025-04-25 00:00:00.000,,,0.0,0.0,Singapore Sportshub,"CHIU, YU-TING"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,Tan,Bernice,nan,SIN,F,2000-11-16 00:00:00.000,25.0,104,Lacticbuds,Run,...,Open,85th Singapore Open Track & Field,2025-04-24 00:00:00.000,2025-04-25 00:00:00.000,,,10000.0,0.0,Singapore Sportshub,"Tan, Bernice"
452,Goh,Shing Ling,nan,SIN,F,1999-06-07 00:00:00.000,26.0,252,TeamFabian,Run,...,Open,85th Singapore Open Track & Field,2025-04-24 00:00:00.000,2025-04-25 00:00:00.000,,,10000.0,0.0,Singapore Sportshub,"Goh, Shing Ling"
453,,,nan,nan,X,nan,nan,0,Club Zoom,Relay,...,Open,85th Singapore Open Track & Field,2025-04-24 00:00:00.000,2025-04-25 00:00:00.000,,,1600.0,0.0,Singapore Sportshub,"{'341': 'Marcus, Seah', '90': 'Irdina, Erza', ..."
454,,,nan,nan,X,nan,nan,0,Erovra Club,Relay,...,Open,85th Singapore Open Track & Field,2025-04-24 00:00:00.000,2025-04-25 00:00:00.000,,,1600.0,0.0,Singapore Sportshub,"{'350': 'CHANG, SZE XIAN LAWRENCE', '271': 'PE..."


In [300]:
# Extract date & year

df_filtered['Meet_start'] = pd.to_datetime(df_filtered['Meet_start'], format='mixed')
df_filtered['Meet_end'] = pd.to_datetime(df_filtered['Meet_end'], format='mixed')

df_filtered['YEAR'] = df_filtered['Meet_start'].dt.year

df_filtered['DATE'] = df_filtered['Meet_end']

#df_filtered['DATE'] = str(df_filtered['Meet_start'].dt.month[1]) + '/' + str(df_filtered['Meet_start'].dt.day[1]) + ' to ' + str(df_filtered['Meet_end'].dt.month[1]) + '/' + str(df_filtered['Meet_end'].dt.day[1]) 
#df_filtered.drop('Meet_start', axis=1, inplace=True)

In [301]:
df_filtered['DATE']

0     2025-04-25
1     2025-04-25
2     2025-04-25
3     2025-04-25
4     2025-04-25
         ...    
451   2025-04-25
452   2025-04-25
453   2025-04-25
454   2025-04-25
455   2025-04-25
Name: DATE, Length: 456, dtype: datetime64[ns]

In [302]:
df_filtered['DATE'] = df_filtered['DATE'].dt.tz_localize('UTC')

In [303]:
#df_filtered['DATE'] = pd.to_datetime(df_filtered['DATE'], dayfirst=True, utc=True)

In [304]:
df_filtered['DATE']

0     2025-04-25 00:00:00+00:00
1     2025-04-25 00:00:00+00:00
2     2025-04-25 00:00:00+00:00
3     2025-04-25 00:00:00+00:00
4     2025-04-25 00:00:00+00:00
                 ...           
451   2025-04-25 00:00:00+00:00
452   2025-04-25 00:00:00+00:00
453   2025-04-25 00:00:00+00:00
454   2025-04-25 00:00:00+00:00
455   2025-04-25 00:00:00+00:00
Name: DATE, Length: 456, dtype: datetime64[ns, UTC]

In [305]:
'''
df_filtered['NOW'] = datetime.datetime.now()

timezone = pytz.timezone('UTC')

df_filtered['NOW'] = datetime.datetime.now().replace(tzinfo=timezone)

df_filtered['delta_time'] = df_filtered['NOW'] - df_filtered['DATE']
'''

"\ndf_filtered['NOW'] = datetime.datetime.now()\n\ntimezone = pytz.timezone('UTC')\n\ndf_filtered['NOW'] = datetime.datetime.now().replace(tzinfo=timezone)\n\ndf_filtered['delta_time'] = df_filtered['NOW'] - df_filtered['DATE']\n"

In [306]:
df_filtered.columns

Index(['Last_name', 'First_name', 'Pref_name', 'Citizen_of', 'Event_gender',
       'Birth_date', 'Ath_age', 'Comp_no', 'Team_name.1', 'Event_name',
       'ActualSeed_timeDisplay', 'Rnd_ltr', 'Reg_no', 'Res_heat',
       'Res_markDisplay', 'Res_wind', 'Res_place', 'Res_lane', 'Res_note',
       'Ath_no', 'Div_name', 'Meet_name', 'Meet_start', 'Meet_end',
       'MultiSubEvent_name', 'Event_note', 'Event_dist', 'Event_score',
       'Meet_location', 'Name', 'YEAR', 'DATE'],
      dtype='object')

In [307]:
df_filtered.drop(['Meet_start', 'Meet_end'], axis=1, inplace=True)


In [308]:
df_filtered.to_csv('column_check.csv', sep=',', encoding='utf-8-sig', index=False)

In [309]:
df_filtered.columns

Index(['Last_name', 'First_name', 'Pref_name', 'Citizen_of', 'Event_gender',
       'Birth_date', 'Ath_age', 'Comp_no', 'Team_name.1', 'Event_name',
       'ActualSeed_timeDisplay', 'Rnd_ltr', 'Reg_no', 'Res_heat',
       'Res_markDisplay', 'Res_wind', 'Res_place', 'Res_lane', 'Res_note',
       'Ath_no', 'Div_name', 'Meet_name', 'MultiSubEvent_name', 'Event_note',
       'Event_dist', 'Event_score', 'Meet_location', 'Name', 'YEAR', 'DATE'],
      dtype='object')

In [310]:
# New schema

# Rename columns. Event Class = 500g for javelin etc (mapped to Event_note). SUB_EVENT=dash under decathlon
# Need to add: Last_name, First_name, Pref_name


df_filtered.columns = ['LAST_NAME', 'FIRST_NAME', 'OTHER_NAME', 'NATIONALITY', 'GENDER', 'DOB', 'AGE', 'TAG_ID', 'TEAM', 'EVENT', 'SEED', 'STAGE', 'UNIQUE_ID', 'HEAT', 'RESULT', 'WIND', 'RANK', 'LANE', 'REMARKS', 'ATHLETE_ID', 'DIVISION', 'COMPETITION', 'SUB_EVENT', 'EVENT_CLASS', 'DISTANCE', 'POINTS', 'VENUE', 'NAME', 'YEAR', 'DATE']

df_filtered['QUALIFICATION']=''
#df_filtered['UNIQUE_ID']=''
df_filtered['DICT_RESULTS']=''
#df_filtered['COMPETITION']='84th Singapore Open'
df_filtered['REGION']='Local'
df_filtered['GROUP']=''
df_filtered['CATEGORY_EVENT']=''
df_filtered['SOURCE']='Meet Manager'
df_filtered['TIMESTAMP']=''
df_filtered['SESSION']=''
df_filtered['HOST_CITY']='Singapore'
df_filtered['RX_TIME']=''


# Map miscellaneous fields to final formats                            
                            
mask = df_filtered['STAGE'].str.contains(r'F', na=True)
df_filtered.loc[mask, 'STAGE'] = 'Final'

mask = df_filtered['STAGE'].str.contains(r'P', na=True)
df_filtered.loc[mask, 'STAGE'] = 'Preliminaries'
                                    
mask = df_filtered['GENDER'].isin(['F'])
df_filtered.loc[mask, 'GENDER'] = 'Female'

mask = df_filtered['GENDER'].isin(['M'])
df_filtered.loc[mask, 'GENDER'] = 'Male'

mask = df_filtered['GENDER'].isin(['X'])
df_filtered.loc[mask, 'GENDER'] = 'Mixed'


# Map general event category 

df_filtered.loc[((df_filtered['DISTANCE'].str.contains(r'60', na=True))&(df_filtered['EVENT'].str.contains(r'Dash', na=True))), 'CATEGORY_EVENT'] = 'Sprint'
df_filtered.loc[((df_filtered['DISTANCE'].str.contains(r'80', na=True))&(df_filtered['EVENT'].str.contains(r'Dash', na=True))), 'CATEGORY_EVENT'] = 'Sprint'
df_filtered.loc[((df_filtered['DISTANCE'].str.contains(r'100', na=True))&(df_filtered['EVENT'].str.contains(r'Dash', na=True))), 'CATEGORY_EVENT'] = 'Sprint'
df_filtered.loc[((df_filtered['DISTANCE'].str.contains(r'200', na=True))&(df_filtered['EVENT'].str.contains(r'Dash', na=True))), 'CATEGORY_EVENT'] = 'Sprint'
df_filtered.loc[((df_filtered['DISTANCE'].str.contains(r'400', na=True))&(df_filtered['EVENT'].str.contains(r'Dash', na=True))), 'CATEGORY_EVENT'] = 'Sprint'


df_filtered.loc[((df_filtered['DISTANCE'].str.contains(r'800', na=True))&(df_filtered['EVENT'].str.contains(r'Run', na=True))), 'CATEGORY_EVENT'] = 'Mid'
df_filtered.loc[((df_filtered['DISTANCE'].str.contains(r'1500', na=True))&(df_filtered['EVENT'].str.contains(r'Run', na=True))), 'CATEGORY_EVENT'] = 'Mid'
df_filtered.loc[((df_filtered['DISTANCE'].str.contains(r'2000', na=True))&(df_filtered['EVENT'].str.contains(r'Run', na=True))), 'CATEGORY_EVENT'] = 'Mid'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Mile', na=True), 'CATEGORY_EVENT'] = 'Mid'


df_filtered.loc[((df_filtered['DISTANCE'].str.contains(r'3000', na=True))&(df_filtered['EVENT'].str.contains(r'Run', na=True))), 'CATEGORY_EVENT'] = 'Long'
df_filtered.loc[((df_filtered['DISTANCE'].str.contains(r'5000', na=True))&(df_filtered['EVENT'].str.contains(r'Run', na=True))), 'CATEGORY_EVENT'] = 'Long'
df_filtered.loc[((df_filtered['DISTANCE'].str.contains(r'10000', na=True))&(df_filtered['EVENT'].str.contains(r'Run', na=True))), 'CATEGORY_EVENT'] = 'Long'


df_filtered.loc[((df_filtered['DISTANCE'].str.contains(r'100', na=True))&(df_filtered['EVENT'].str.contains(r'Hurdes', na=True))), 'CATEGORY_EVENT'] = 'Hurdles'
df_filtered.loc[((df_filtered['DISTANCE'].str.contains(r'110', na=True))&(df_filtered['EVENT'].str.contains(r'Hurdes', na=True))), 'CATEGORY_EVENT'] = 'Hurdles'
df_filtered.loc[((df_filtered['DISTANCE'].str.contains(r'400', na=True))&(df_filtered['EVENT'].str.contains(r'Hurdles', na=True))), 'CATEGORY_EVENT'] = 'Hurdles'


# Override values

df_filtered.loc[df_filtered['EVENT'].str.contains(r'Discus', na=True), 'CATEGORY_EVENT'] = 'Throw'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Javelin', na=True), 'CATEGORY_EVENT'] = 'Throw'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Shot', na=True), 'CATEGORY_EVENT'] = 'Throw'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Vault', na=True), 'CATEGORY_EVENT'] = 'Jump'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Throw', na=True), 'CATEGORY_EVENT'] = 'Throw'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Jump', na=True), 'CATEGORY_EVENT'] = 'Jump'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Relay', na=True), 'CATEGORY_EVENT'] = 'Relay'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Steeple', na=True), 'CATEGORY_EVENT'] = 'Steeple'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Walk', na=True), 'CATEGORY_EVENT'] = 'Walk'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Hurdles', na=True), 'CATEGORY_EVENT'] = 'Hurdles'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Pentathlon', na=True), 'CATEGORY_EVENT'] = 'Pentathlon'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Triathlon', na=True), 'CATEGORY_EVENT'] = 'Triathlon'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Decathlon', na=True), 'CATEGORY_EVENT'] = 'Decathlon'


# Rename conventions e.g. 100 Meters becomes 100m to be consistent with rest of dataset  

#df_filtered.loc[df_filtered['EVENT'].str.contains(r'100.0 Dash', na=True), 'EVENT'] = '100m'

    

df_filtered = df_filtered.reindex(columns= ['FIRST_NAME', 'LAST_NAME', 'OTHER_NAME', 'NAME', 'RANK', 'TAG_ID', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION',
                                        'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 'POINTS', 'AGE', 'GENDER', 'UNIQUE_ID', 'NATIONALITY',
                                        'DICT_RESULTS', 'YEAR', 'DATE', 'COMPETITION', 'REGION', 'DOB', 'GROUP', 'CATEGORY_EVENT', 'ATHLETE_ID',
                                        'SOURCE', 'REMARKS', 'TIMESTAMP', 'VENUE', 'SUB_EVENT', 'SESSION', 'EVENT_CLASS', 'DISTANCE', 'HOST_CITY', 'RX_TIME'])




In [311]:
df_filtered

,FIRST_NAME,LAST_NAME,OTHER_NAME,NAME,RANK,TAG_ID,TEAM,SEED,RESULT,QUALIFICATION,...,SOURCE,REMARKS,TIMESTAMP,VENUE,SUB_EVENT,SESSION,EVENT_CLASS,DISTANCE,HOST_CITY,RX_TIME
0,Emilia,Nova,nan,"Nova, Emilia",5,37,DKI Java,36.46m,37.15m,,...,Meet Manager,,,Singapore Sportshub,,,,0.0,Singapore,
1,Siti Nazirah,Napisatul Munawaroh,nan,"Napisatul Munawaroh, Siti Nazirah",4,36,DKI Java,50.69m,45.55m,,...,Meet Manager,,,Singapore Sportshub,,,,0.0,Singapore,
2,Attina Nurkamila Int,Bahtiar,nan,"Bahtiar, Attina Nurkamila Int",3,83,Indonesia,52.56m,46.31m,,...,Meet Manager,nan,,Singapore Sportshub,,,,0.0,Singapore,
3,MINJI,KIM,nan,"KIM, MINJI",2,236,South Korea,53.15m,47.65m,,...,Meet Manager,nan,,Singapore Sportshub,,,,0.0,Singapore,
4,YU-TING,CHIU,nan,"CHIU, YU-TING",1,3,Chinese Taipei,52.26m,53.13m,,...,Meet Manager,nan,,Singapore Sportshub,,,,0.0,Singapore,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,Bernice,Tan,nan,"Tan, Bernice",3,104,Lacticbuds,45:02.00,44:25.61,,...,Meet Manager,nan,,Singapore Sportshub,,,,10000.0,Singapore,
452,Shing Ling,Goh,nan,"Goh, Shing Ling",2,252,TeamFabian,38:44.00,39:21.58,,...,Meet Manager,nan,,Singapore Sportshub,,,,10000.0,Singapore,
453,,,nan,"{'341': 'Marcus, Seah', '90': 'Irdina, Erza', ...",3,0,Club Zoom,,4:12.76,,...,Meet Manager,nan,,Singapore Sportshub,,,,1600.0,Singapore,
454,,,nan,"{'350': 'CHANG, SZE XIAN LAWRENCE', '271': 'PE...",1,0,Erovra Club,,3:53.02,,...,Meet Manager,nan,,Singapore Sportshub,,,,1600.0,Singapore,


In [312]:
df_filtered['DOB']=df_filtered.DOB.str.rstrip('00:00:00.000')


In [313]:
df_filtered

,FIRST_NAME,LAST_NAME,OTHER_NAME,NAME,RANK,TAG_ID,TEAM,SEED,RESULT,QUALIFICATION,...,SOURCE,REMARKS,TIMESTAMP,VENUE,SUB_EVENT,SESSION,EVENT_CLASS,DISTANCE,HOST_CITY,RX_TIME
0,Emilia,Nova,nan,"Nova, Emilia",5,37,DKI Java,36.46m,37.15m,,...,Meet Manager,,,Singapore Sportshub,,,,0.0,Singapore,
1,Siti Nazirah,Napisatul Munawaroh,nan,"Napisatul Munawaroh, Siti Nazirah",4,36,DKI Java,50.69m,45.55m,,...,Meet Manager,,,Singapore Sportshub,,,,0.0,Singapore,
2,Attina Nurkamila Int,Bahtiar,nan,"Bahtiar, Attina Nurkamila Int",3,83,Indonesia,52.56m,46.31m,,...,Meet Manager,nan,,Singapore Sportshub,,,,0.0,Singapore,
3,MINJI,KIM,nan,"KIM, MINJI",2,236,South Korea,53.15m,47.65m,,...,Meet Manager,nan,,Singapore Sportshub,,,,0.0,Singapore,
4,YU-TING,CHIU,nan,"CHIU, YU-TING",1,3,Chinese Taipei,52.26m,53.13m,,...,Meet Manager,nan,,Singapore Sportshub,,,,0.0,Singapore,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,Bernice,Tan,nan,"Tan, Bernice",3,104,Lacticbuds,45:02.00,44:25.61,,...,Meet Manager,nan,,Singapore Sportshub,,,,10000.0,Singapore,
452,Shing Ling,Goh,nan,"Goh, Shing Ling",2,252,TeamFabian,38:44.00,39:21.58,,...,Meet Manager,nan,,Singapore Sportshub,,,,10000.0,Singapore,
453,,,nan,"{'341': 'Marcus, Seah', '90': 'Irdina, Erza', ...",3,0,Club Zoom,,4:12.76,,...,Meet Manager,nan,,Singapore Sportshub,,,,1600.0,Singapore,
454,,,nan,"{'350': 'CHANG, SZE XIAN LAWRENCE', '271': 'PE...",1,0,Erovra Club,,3:53.02,,...,Meet Manager,nan,,Singapore Sportshub,,,,1600.0,Singapore,


In [314]:
df_filtered['POINTS']

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
451    0.0
452    0.0
453    0.0
454    0.0
455    0.0
Name: POINTS, Length: 456, dtype: object

In [315]:
df_filtered['RESULT']

0        37.15m
1        45.55m
2        46.31m
3        47.65m
4        53.13m
         ...   
451    44:25.61
452    39:21.58
453     4:12.76
454     3:53.02
455     3:56.33
Name: RESULT, Length: 456, dtype: object

In [333]:
# Load formatted file post processing

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/mm_files/2025/')

df_filtered = pd.read_csv("Processed-GenericData-85th Singapore Open Track - Field-24Apr2025-002_missing.csv")



In [334]:
df_filtered

,FIRST_NAME,LAST_NAME,OTHER_NAME,NAME,RANK,TAG_ID,TEAM,SEED,RESULT,QUALIFICATION,...,SOURCE,REMARKS,TIMESTAMP,VENUE,SUB_EVENT,SESSION,EVENT_CLASS,DISTANCE,HOST_CITY,RX_TIME
0,Jonathan,Low,NaN,"Low, Jonathan",2,143,National University Singapore,44.41m,42.74m,NaN,...,Meet Manager,NaN,05:45.1,Singapore Sportshub,NaN,NaN,NaN,0,Singapore,NaN
1,Sheik Fayz,Sheik Alau'ddin,NaN,"Sheik Alau'ddin, Sheik Fayz",4,228,Singapore Sports School,41.80m,41.35m,NaN,...,Meet Manager,NaN,05:45.1,Singapore Sportshub,NaN,NaN,NaN,0,Singapore,NaN
2,Jun Jie,Tan,NaN,"Tan, Jun Jie",3,326,Wings Athletics Club,44.41m,41.82m,NaN,...,Meet Manager,NaN,05:45.1,Singapore Sportshub,NaN,NaN,NaN,0,Singapore,NaN
3,Wei De,Toh,NaN,"Toh, Wei De",5,328,Wings Athletics Club,37.13m,36.60m,NaN,...,Meet Manager,NaN,05:45.1,Singapore Sportshub,NaN,NaN,NaN,0,Singapore,NaN
4,Hanchen,Wang,NaN,"Wang, Hanchen",6,80,Hwa Chong Alumni Association,40.43m,35.81m,NaN,...,Meet Manager,NaN,05:45.1,Singapore Sportshub,NaN,NaN,NaN,0,Singapore,NaN
5,ERIC CHUN WAI,YEE,NaN,"YEE, ERIC CHUN WAI",1,213,Singapore,49.51m,47.78m,NaN,...,Meet Manager,NaN,05:45.1,Singapore Sportshub,NaN,NaN,NaN,0,Singapore,NaN


In [335]:
# Remove special characters
    
for col in df_filtered.columns:
    df_filtered[col] = df_filtered[col].astype(str)
    df_filtered[col] = df_filtered[col].str.replace('\xa0', ' ', regex=True)
    df_filtered[col] = df_filtered[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    df_filtered[col] = df_filtered[col].str.replace('\r', ' ', regex=True)
    df_filtered[col] = df_filtered[col].str.replace('\n', ' ', regex=True)
    df_filtered[col] = df_filtered[col].str.strip()



In [336]:
# Set timestamp

from datetime import datetime

ct = datetime.now()

df_filtered.loc[:,'TIMESTAMP'] = ct


In [337]:
df_filtered['TIMESTAMP'] = pd.to_datetime(df_filtered['TIMESTAMP'], format='mixed', dayfirst=False, utc=False)

In [338]:
df_filtered['TIMESTAMP'] = df_filtered['TIMESTAMP'].astype(str)

In [339]:
df_filtered['TIMESTAMP']

0    2025-07-16 10:01:26.214072
1    2025-07-16 10:01:26.214072
2    2025-07-16 10:01:26.214072
3    2025-07-16 10:01:26.214072
4    2025-07-16 10:01:26.214072
5    2025-07-16 10:01:26.214072
Name: TIMESTAMP, dtype: object

In [340]:
df_filtered['DATE']

0    2025-04-25 00:00:00+00:00
1    2025-04-25 00:00:00+00:00
2    2025-04-25 00:00:00+00:00
3    2025-04-25 00:00:00+00:00
4    2025-04-25 00:00:00+00:00
5    2025-04-25 00:00:00+00:00
Name: DATE, dtype: object

In [341]:
# Load BQ Modified Schema

modified_schema = [
            bigquery.SchemaField("FIRST_NAME", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("LAST_NAME", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("OTHER_NAME", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("NAME", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("RANK", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("TAG_ID", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("TEAM", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("SEED", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("RESULT", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("QUALIFICATION", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("HEAT", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("LANE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("WIND", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("EVENT", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("DIVISION", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("STAGE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("POINTS", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("AGE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("GENDER", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("UNIQUE_ID", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("NATIONALITY", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("DICT_RESULTS", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("YEAR", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("DATE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("COMPETITION", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("REGION", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("DOB", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("GROUP", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("CATEGORY_EVENT", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("ATHLETE_ID", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("SOURCE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("REMARKS", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("TIMESTAMP", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("VENUE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("SUB_EVENT", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("SESSION", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("EVENT_CLASS", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("DISTANCE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("HOST_CITY", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("RX_TIME", "STRING", mode="NULLABLE"),
]

In [342]:
df = df_filtered

In [343]:
# Check dataframe before uploading in BQ

gender_pattern = 'Male|Female|Mixed'
year_pattern = '20\d\d'
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
df['YEAR'] = df['YEAR'].astype(str)
df['DOB'] = df['DOB'].astype(str)
df['DATE'] = pd.to_datetime(df['DATE'], format='mixed', dayfirst=False, utc=True)


for index, row in df.iterrows():
        
    assert re.search(gender_pattern, row['GENDER']) is not None   # check format of gender input
    assert row['NAME'] is not None
    assert len(row['NAME'])>5
    assert row['DATE'] is not None
    assert row['EVENT'] is not None 
    assert row['COMPETITION'] is not None 
    assert row['REGION'] is not None
    assert row['DATE'] is not None
    assert row['CATEGORY_EVENT'] is not None
 #   assert ((len(row['DOB'])>3) or (row['DOB'] is None))
    assert re.search(year_pattern, row['YEAR']) is not None   # check format of year
    assert not row['DATE'] is pd.NaT
    
    # Check column locations before loading

    assert len(df.columns) == 40
    assert df.columns.get_loc('RESULT') == 8
    assert df.columns.get_loc('AGE') == 17
    assert df.columns.get_loc('WIND') == 12
    assert df.columns.get_loc('COMPETITION')==24
    

df['DATE'] = df['DATE'].astype(str)  # convert DATE column back to string to conform to schema

print('PASSED')

df

PASSED


,FIRST_NAME,LAST_NAME,OTHER_NAME,NAME,RANK,TAG_ID,TEAM,SEED,RESULT,QUALIFICATION,...,SOURCE,REMARKS,TIMESTAMP,VENUE,SUB_EVENT,SESSION,EVENT_CLASS,DISTANCE,HOST_CITY,RX_TIME
0,Jonathan,Low,nan,"Low, Jonathan",2,143,National University Singapore,44.41m,42.74m,nan,...,Meet Manager,nan,2025-07-16 10:01:26.214072,Singapore Sportshub,nan,nan,nan,0,Singapore,nan
1,Sheik Fayz,Sheik Alau'ddin,nan,"Sheik Alau'ddin, Sheik Fayz",4,228,Singapore Sports School,41.80m,41.35m,nan,...,Meet Manager,nan,2025-07-16 10:01:26.214072,Singapore Sportshub,nan,nan,nan,0,Singapore,nan
2,Jun Jie,Tan,nan,"Tan, Jun Jie",3,326,Wings Athletics Club,44.41m,41.82m,nan,...,Meet Manager,nan,2025-07-16 10:01:26.214072,Singapore Sportshub,nan,nan,nan,0,Singapore,nan
3,Wei De,Toh,nan,"Toh, Wei De",5,328,Wings Athletics Club,37.13m,36.60m,nan,...,Meet Manager,nan,2025-07-16 10:01:26.214072,Singapore Sportshub,nan,nan,nan,0,Singapore,nan
4,Hanchen,Wang,nan,"Wang, Hanchen",6,80,Hwa Chong Alumni Association,40.43m,35.81m,nan,...,Meet Manager,nan,2025-07-16 10:01:26.214072,Singapore Sportshub,nan,nan,nan,0,Singapore,nan
5,ERIC CHUN WAI,YEE,nan,"YEE, ERIC CHUN WAI",1,213,Singapore,49.51m,47.78m,nan,...,Meet Manager,nan,2025-07-16 10:01:26.214072,Singapore Sportshub,nan,nan,nan,0,Singapore,nan


In [344]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 40 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   FIRST_NAME      6 non-null      object        
 1   LAST_NAME       6 non-null      object        
 2   OTHER_NAME      6 non-null      object        
 3   NAME            6 non-null      object        
 4   RANK            6 non-null      object        
 5   TAG_ID          6 non-null      object        
 6   TEAM            6 non-null      object        
 7   SEED            6 non-null      object        
 8   RESULT          6 non-null      object        
 9   QUALIFICATION   6 non-null      object        
 10  HEAT            6 non-null      object        
 11  LANE            6 non-null      object        
 12  WIND            6 non-null      object        
 13  EVENT           6 non-null      object        
 14  DIVISION        6 non-null      object        
 15  STAGE     

In [345]:
df = df.astype(str)

In [347]:
# Write and append to PRODUCTION table in BigQuery directly with schema

client = bigquery.Client()


job_config = bigquery.LoadJobConfig(
    schema=modified_schema,
    write_disposition="WRITE_APPEND",  # or WRITE_APPEND, WRITE_EMPTY
)

job = client.load_table_from_dataframe(df, 'saa-analytics.results.PRODUCTION', job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=saa-analytics, location=US, id=6036c529-85a5-45c3-8ffd-cd78253ea443>

In [322]:
# Save csv file

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/mm_files/2025/')


df_filtered.to_csv('Processed-GenericData-85th Singapore Open Track - Field-24Apr2025-002_complete.csv')

In [448]:
df_filtered.columns

Index(['FIRST_NAME', 'LAST_NAME', 'OTHER_NAME', 'NAME', 'RANK', 'TAG_ID',
       'TEAM', 'SEED', 'RESULT', 'QUALIFICATION', 'HEAT', 'LANE', 'WIND',
       'EVENT', 'DIVISION', 'STAGE', 'POINTS', 'AGE', 'GENDER', 'UNIQUE_ID',
       'NATIONALITY', 'DICT_RESULTS', 'YEAR', 'DATE', 'COMPETITION', 'REGION',
       'DOB', 'GROUP', 'CATEGORY_EVENT', 'ATHLETE_ID', 'SOURCE', 'REMARKS',
       'TIMESTAMP', 'VENUE', 'SUB_EVENT', 'SESSION', 'EVENT_CLASS', 'DISTANCE',
       'HOST_CITY', 'RX_TIME'],
      dtype='object')

In [348]:
### Process GenericData-84th Singapore Open Track - Field-18Apr2024-001.mdb file due to different column names

In [264]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/mm_files/2024/')

df = pd.read_csv("GenericData-84th Singapore Open Track - Field-18Apr202.csv", low_memory=False)

df.shape[1]

176

In [265]:
list(df.columns)

['EVENT_NUMLTR',
 'Rnd_ltr',
 'MultiSubEvent_no',
 'TRK_FIELD',
 'Ind_rel',
 'EVENT_GENDER',
 'EVENT_DIST',
 'EVENT_CODE',
 'EVENT_NAME',
 'FULL_EVENTNAME',
 'MultiSubEvent_name',
 'low_age',
 'high_age',
 'Div_no',
 'Event_note',
 'Event_Type',
 'Entry_fee',
 'Ath_no',
 'COMP_NO',
 'LAST_NAME',
 'FIRST_NAME',
 'Reg_no',
 'Team_no',
 'TEAM_ABBR',
 'Team_lsc',
 'Relay_Ltr',
 'Event_score',
 'Res_heat',
 'Res_lane',
 'Res_stat',
 'RES_MARKDISPLAY',
 'RES_MARK',
 'RES_MEAS',
 'Res_handtime',
 'RES_WIND',
 'Res_agegradeDisplay',
 'Res_agegrade',
 'Res_percent',
 'RES_HEATPLACE',
 'RES_PLACE',
 'Res_exh',
 'Res_note',
 'RelayLeg1_Ath_no',
 'RelayLeg1_Last_name',
 'RelayLeg1_First_name',
 'RelayLeg2_Ath_no',
 'RelayLeg2_Last_name',
 'RelayLeg2_First_name',
 'RelayLeg3_Ath_no',
 'RelayLeg3_Last_name',
 'RelayLeg3_First_name',
 'RelayLeg4_Ath_no',
 'RelayLeg4_Last_name',
 'RelayLeg4_First_name',
 'RelayLeg5_Ath_no',
 'RelayLeg5_Last_name',
 'RelayLeg5_First_name',
 'RelayLeg6_Ath_no',
 'RelayL

In [25]:
df_filtered = df[['LAST_NAME', 'FIRST_NAME', 'EVENT_GENDER', 'BIRTH_DATE', 'ATH_AGE', 'COMP_NO', 'TEAM_NAME.1','FULL_EVENTNAME', 'ACTUALSEED_TIMEDISPLAY','Rnd_ltr',
                 'Res_heat', 'Res_lane', 'RES_MARKDISPLAY', 'RES_WIND', 'RES_PLACE', 'Res_note', 'Ath_no', 'Div_name', 'Relay_Ltr', 'RelayLeg1_Ath_no', 'RelayLeg1_Last_name', 'RelayLeg1_First_name',
                 'RelayLeg2_Ath_no', 'RelayLeg2_Last_name', 'RelayLeg2_First_name', 'RelayLeg3_Ath_no', 'RelayLeg3_Last_name', 'RelayLeg3_First_name', 'RelayLeg4_Ath_no', 'RelayLeg4_Last_name', 'RelayLeg4_First_name',
                 'RelayLeg5_Ath_no', 'RelayLeg5_Last_name', 'RelayLeg5_First_name', 'RelayLeg6_Ath_no', 'RelayLeg6_Last_name', 'RelayLeg6_First_name', 'RelayLeg7_Ath_no', 'RelayLeg7_Last_name', 'RelayLeg7_First_name',
                 'RelayLeg8_Ath_no', 'RelayLeg8_Last_name', 'RelayLeg8_First_name', 'Meet_name', 'Meet_start']].copy()


In [26]:
df_filtered.drop_duplicates(inplace=True)    


In [27]:
df_filtered['Name']= df_filtered['LAST_NAME'] + ', '+ df_filtered['FIRST_NAME']

df_filtered.drop(['LAST_NAME', 'FIRST_NAME'], axis=1, inplace=True)


In [28]:
df_filtered['Relay1_Leg_Name']= df_filtered['RelayLeg1_Last_name'] + ', '+ df_filtered['RelayLeg1_First_name']
df_filtered['Relay2_Leg_Name']= df_filtered['RelayLeg2_Last_name'] + ', '+ df_filtered['RelayLeg2_First_name']
df_filtered['Relay3_Leg_Name']= df_filtered['RelayLeg3_Last_name'] + ', '+ df_filtered['RelayLeg3_First_name']
df_filtered['Relay4_Leg_Name']= df_filtered['RelayLeg4_Last_name'] + ', '+ df_filtered['RelayLeg4_First_name']
df_filtered['Relay5_Leg_Name']= df_filtered['RelayLeg5_Last_name'] + ', '+ df_filtered['RelayLeg5_First_name']
df_filtered['Relay6_Leg_Name']= df_filtered['RelayLeg6_Last_name'] + ', '+ df_filtered['RelayLeg6_First_name']
df_filtered['Relay7_Leg_Name']= df_filtered['RelayLeg7_Last_name'] + ', '+ df_filtered['RelayLeg7_First_name']
df_filtered['Relay8_Leg_Name']= df_filtered['RelayLeg8_Last_name'] + ', '+ df_filtered['RelayLeg8_First_name']

df_filtered.drop(['RelayLeg1_Last_name', 'RelayLeg1_First_name', 'RelayLeg2_Last_name', 'RelayLeg2_First_name',
                 'RelayLeg3_Last_name', 'RelayLeg3_First_name', 'RelayLeg4_Last_name', 'RelayLeg4_First_name',
                 'RelayLeg5_Last_name', 'RelayLeg5_First_name', 'RelayLeg6_Last_name', 'RelayLeg6_First_name',
                 'RelayLeg7_Last_name', 'RelayLeg7_First_name', 'RelayLeg8_Last_name', 'RelayLeg8_First_name'], axis=1, inplace=True)


In [29]:
df_filtered.reset_index(inplace = True, drop = True)  # reset index after dropping duplicates


In [30]:
# create dict for relay names

for i in range(len(df_filtered)):
    
    list_no=[]
    list_names=[]
    
    if 'Relay' in df_filtered.loc[i, 'FULL_EVENTNAME']:
                
        list_no.append(df_filtered.loc[i, 'RelayLeg1_Ath_no'])
        list_no.append(df_filtered.loc[i, 'RelayLeg2_Ath_no'])
        list_no.append(df_filtered.loc[i, 'RelayLeg3_Ath_no'])
        list_no.append(df_filtered.loc[i, 'RelayLeg4_Ath_no'])
        list_no.append(df_filtered.loc[i, 'RelayLeg5_Ath_no'])
        list_no.append(df_filtered.loc[i, 'RelayLeg6_Ath_no'])
        list_no.append(df_filtered.loc[i, 'RelayLeg7_Ath_no'])
        list_no.append(df_filtered.loc[i, 'RelayLeg8_Ath_no'])
        
        
        list_names.append(df_filtered.loc[i, 'Relay1_Leg_Name'])
        list_names.append(df_filtered.loc[i, 'Relay2_Leg_Name'])
        list_names.append(df_filtered.loc[i, 'Relay3_Leg_Name'])
        list_names.append(df_filtered.loc[i, 'Relay4_Leg_Name'])
        list_names.append(df_filtered.loc[i, 'Relay5_Leg_Name'])
        list_names.append(df_filtered.loc[i, 'Relay6_Leg_Name'])
        list_names.append(df_filtered.loc[i, 'Relay7_Leg_Name'])
        list_names.append(df_filtered.loc[i, 'Relay8_Leg_Name'])
        
        
        list_no = [i for i in list_no if i != 0]
        
        list_names = [i for i in list_names if i != ' ,  ']   
                
        res = {}
        for key in list_no:
            for value in list_names:
                res[key] = value
                list_names.remove(value)
                
                break
        
        df_filtered.at[i, 'Name'] = res  # must use .at for dict
                           
                



In [31]:
df_filtered.drop(['Relay_Ltr', 'RelayLeg1_Ath_no', 'RelayLeg2_Ath_no', 'RelayLeg3_Ath_no', 'RelayLeg4_Ath_no', 
                  'RelayLeg5_Ath_no', 'RelayLeg6_Ath_no', 'RelayLeg7_Ath_no', 'RelayLeg8_Ath_no', 'Relay1_Leg_Name',
                  'Relay2_Leg_Name', 'Relay3_Leg_Name', 'Relay4_Leg_Name', 'Relay5_Leg_Name', 'Relay6_Leg_Name',
                  'Relay7_Leg_Name', 'Relay8_Leg_Name'], axis=1, inplace=True)


In [32]:
df_filtered['Name']=df_filtered['Name'].astype(str) # convert dict to str so duplicates can be dropped


In [33]:
# Extract date & year

df_filtered['Meet_start'] = pd.to_datetime(df_filtered['Meet_start'])

df_filtered['DATE'] = df_filtered['Meet_start'].dt.year

df_filtered.drop('Meet_start', axis=1, inplace=True)

In [34]:
df_filtered.columns = ['GENDER', 'DOB', 'AGE', 'TAG_ID', 'TEAM', 'EVENT', 'SEED', 'STAGE', 'HEAT', 'LANE', 'RESULT',
                      'WIND', 'RANK', 'REMARKS', 'ATHLETE_ID', 'DIVISION', 'COMPETITION', 'NAME', 'DATE']

df_filtered['QUALIFICATION']=''
df_filtered['POINTS']=''
df_filtered['UNIQUE_ID']=''
df_filtered['COUNTRY']=''
df_filtered['DICT_RESULTS']=''
#df_filtered['COMPETITION']='84th Singapore Open'
df_filtered['REGION']='Local'
df_filtered['GROUP']=''
df_filtered['CATEGORY_EVENT']=''
df_filtered['SOURCE']='Meet Manager'
df_filtered['TIMESTAMP']=''
df_filtered['FREE_FIELD']=''
df_filtered['FREE_FIELD2']=''
df_filtered['FREE_FIELD3']=''

# Map miscellaneous fields to final formats                            
                            
mask = df_filtered['STAGE'].str.contains(r'F', na=True)
df_filtered.loc[mask, 'STAGE'] = 'Final'

mask = df_filtered['STAGE'].str.contains(r'P', na=True)
df_filtered.loc[mask, 'STAGE'] = 'Preliminaries'
                                    
mask = df_filtered['GENDER'].isin(['F'])
df_filtered.loc[mask, 'GENDER'] = 'Female'

mask = df_filtered['GENDER'].isin(['M'])
df_filtered.loc[mask, 'GENDER'] = 'Male'

# Map general event category 

df_filtered.loc[df_filtered['EVENT'].str.contains(r'Vault', na=True), 'CATEGORY_EVENT'] = 'Jump'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Throw', na=True), 'CATEGORY_EVENT'] = 'Throw'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'100', na=True), 'CATEGORY_EVENT'] = 'Sprint'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Javelin', na=True), 'CATEGORY_EVENT'] = 'Throw'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Shot', na=True), 'CATEGORY_EVENT'] = 'Throw'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'400', na=True), 'CATEGORY_EVENT'] = 'Sprint'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'600', na=True), 'CATEGORY_EVENT'] = 'Mid'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'200', na=True), 'CATEGORY_EVENT'] = 'Sprint'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'50', na=True), 'CATEGORY_EVENT'] = 'Sprint'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'80', na=True), 'CATEGORY_EVENT'] = 'Sprint'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'300', na=True), 'CATEGORY_EVENT'] = 'Sprint'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'3000', na=True), 'CATEGORY_EVENT'] = 'Long'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Mile', na=True), 'CATEGORY_EVENT'] = 'Mid'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'5000', na=True), 'CATEGORY_EVENT'] = 'Long'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'2000', na=True), 'CATEGORY_EVENT'] = 'Mid'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'800', na=True), 'CATEGORY_EVENT'] = 'Mid'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'1500', na=True), 'CATEGORY_EVENT'] = 'Mid'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Discus', na=True), 'CATEGORY_EVENT'] = 'Throw'

# Override values

df_filtered.loc[df_filtered['EVENT'].str.contains(r'Jump', na=True), 'CATEGORY_EVENT'] = 'Jump'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Relay', na=True), 'CATEGORY_EVENT'] = 'Relay'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Steeple', na=True), 'CATEGORY_EVENT'] = 'Steeple'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Walk', na=True), 'CATEGORY_EVENT'] = 'Walk'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Hurdles', na=True), 'CATEGORY_EVENT'] = 'Hurdles'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Pentathlon', na=True), 'CATEGORY_EVENT'] = 'Pentathlon'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Triathlon', na=True), 'CATEGORY_EVENT'] = 'Triathlon'

 
    

df_filtered = df_filtered.reindex(columns= ['RANK', 'TAG_ID', 'NAME', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION', 'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 
                                  'POINTS', 'AGE', 'GENDER', 'UNIQUE_ID', 'COUNTRY', 'DICT_RESULTS', 'DATE', 'COMPETITION', 'REGION', 'DOB','GROUP', 'CATEGORY_EVENT', 
                                      'ATHLETE_ID', 'SOURCE', 'REMARKS', 'TIMESTAMP', 'FREE_FIELD', 'FREE_FIELD2', 'FREE_FIELD3'])


In [399]:
# Add timestamp for loading

ct = datetime.datetime.now()

df_filtered.loc[:,'TIMESTAMP'] = ct


In [35]:
df_filtered.to_csv('Processed -84th Singapore Open Track - Field-18Apr2024-001.mdb_202405041853.csv', sep=',', encoding='utf-8-sig', index=False)

In [141]:
## Latest version with expanded data schema including venue, date, session etc.
####
#  Process Pesta Sukan Data
####

In [50]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/mm_files/2025/')

df = pd.read_csv("GenericData-SSP Invitational U15 and U18 2025-31May2025-001.csv", low_memory=False)

df.shape[1]

176

In [51]:
df = df.astype(str)

In [52]:
# Use boolean masking to update full_eventname field for tri/pent events
# This is the only additional step for triathlon/pentathlon events

df.loc[df['Event_name'] == 'Triathlon', 'Full_Eventname'] = 'Triathlon'+ ' - ' + df['Event_dist'] +' '+df['MultiSubEvent_name']
df.loc[df['Event_name'] == 'Outdoor Pentathlon', 'Full_Eventname'] = 'Outdoor Pentathlon'+ ' - ' + df['Event_dist'] + ' '+ df['MultiSubEvent_name']


In [53]:
df

,Event_numltr,Rnd_ltr,MultiSubEvent_no,Trk_Field,Ind_rel,Event_gender,Event_dist,Event_code,Event_name,Full_Eventname,...,Div_name.1,Team_region,Reg_abbr,Reg_name,Meet_name,Meet_header1,Meet_header2,Meet_location,Meet_start,Meet_end
0,1,F,0,T,I,F,100.0,A,Dash,Girls 13-14 100 Meter Dash U15,...,,0,,,SSP Invitational U15 and U18 2025,nan,Singapore Sports School,Singapore Sports School,2025-05-31 00:00:00.000,2025-05-31 00:00:00.000
1,1,P,0,T,I,F,100.0,A,Dash,Girls 13-14 100 Meter Dash U15,...,,0,,,SSP Invitational U15 and U18 2025,nan,Singapore Sports School,Singapore Sports School,2025-05-31 00:00:00.000,2025-05-31 00:00:00.000
2,1,P,0,T,I,F,100.0,A,Dash,Girls 13-14 100 Meter Dash U15,...,,0,,,SSP Invitational U15 and U18 2025,nan,Singapore Sports School,Singapore Sports School,2025-05-31 00:00:00.000,2025-05-31 00:00:00.000
3,1,F,0,T,I,F,100.0,A,Dash,Girls 13-14 100 Meter Dash U15,...,,0,,,SSP Invitational U15 and U18 2025,nan,Singapore Sports School,Singapore Sports School,2025-05-31 00:00:00.000,2025-05-31 00:00:00.000
4,1,P,0,T,I,F,100.0,A,Dash,Girls 13-14 100 Meter Dash U15,...,,0,,,SSP Invitational U15 and U18 2025,nan,Singapore Sports School,Singapore Sports School,2025-05-31 00:00:00.000,2025-05-31 00:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,61,F,0,F,I,F,0.0,M,Long Jump,Women 16-99 Long Jump Open,...,,0,,,SSP Invitational U15 and U18 2025,nan,Singapore Sports School,Singapore Sports School,2025-05-31 00:00:00.000,2025-05-31 00:00:00.000
291,61,F,0,F,I,F,0.0,M,Long Jump,Women 16-99 Long Jump Open,...,,0,,,SSP Invitational U15 and U18 2025,nan,Singapore Sports School,Singapore Sports School,2025-05-31 00:00:00.000,2025-05-31 00:00:00.000
292,62,F,0,F,I,M,0.0,K,High Jump,Men 16-99 High Jump Open,...,,0,,,SSP Invitational U15 and U18 2025,nan,Singapore Sports School,Singapore Sports School,2025-05-31 00:00:00.000,2025-05-31 00:00:00.000
293,62,F,0,F,I,M,0.0,K,High Jump,Men 16-99 High Jump Open,...,,0,,,SSP Invitational U15 and U18 2025,nan,Singapore Sports School,Singapore Sports School,2025-05-31 00:00:00.000,2025-05-31 00:00:00.000


In [54]:
df_filtered = df[['Last_name', 'First_name', 'Event_gender', 'Birth_date', 'Ath_age', 'Comp_no', 'Team_name.1','Full_Eventname', 'ActualSeed_timeDisplay','Rnd_ltr',
                 'Res_heat', 'Res_lane', 'Res_markDisplay', 'Res_wind', 'Res_place', 'Res_note', 'Ath_no', 'Div_name', 'Relay_Ltr', 'RelayLeg1_Ath_no', 'RelayLeg1_Last_name', 'RelayLeg1_First_name',
                 'RelayLeg2_Ath_no', 'RelayLeg2_Last_name', 'RelayLeg2_First_name', 'RelayLeg3_Ath_no', 'RelayLeg3_Last_name', 'RelayLeg3_First_name', 'RelayLeg4_Ath_no', 'RelayLeg4_Last_name', 'RelayLeg4_First_name',
                 'RelayLeg5_Ath_no', 'RelayLeg5_Last_name', 'RelayLeg5_First_name', 'RelayLeg6_Ath_no', 'RelayLeg6_Last_name', 'RelayLeg6_First_name', 'RelayLeg7_Ath_no', 'RelayLeg7_Last_name', 'RelayLeg7_First_name',
                 'RelayLeg8_Ath_no', 'RelayLeg8_Last_name', 'RelayLeg8_First_name', 'Meet_name', 'Meet_start']].copy()


In [55]:
df_filtered.drop_duplicates(inplace=True)    


In [56]:
df_filtered['Name']= df_filtered['Last_name'] + ', '+ df_filtered['First_name']

df_filtered.drop(['Last_name', 'First_name'], axis=1, inplace=True)


In [57]:
df_filtered['Relay1_Leg_Name']= df_filtered['RelayLeg1_Last_name'] + ', '+ df_filtered['RelayLeg1_First_name']
df_filtered['Relay2_Leg_Name']= df_filtered['RelayLeg2_Last_name'] + ', '+ df_filtered['RelayLeg2_First_name']
df_filtered['Relay3_Leg_Name']= df_filtered['RelayLeg3_Last_name'] + ', '+ df_filtered['RelayLeg3_First_name']
df_filtered['Relay4_Leg_Name']= df_filtered['RelayLeg4_Last_name'] + ', '+ df_filtered['RelayLeg4_First_name']
df_filtered['Relay5_Leg_Name']= df_filtered['RelayLeg5_Last_name'] + ', '+ df_filtered['RelayLeg5_First_name']
df_filtered['Relay6_Leg_Name']= df_filtered['RelayLeg6_Last_name'] + ', '+ df_filtered['RelayLeg6_First_name']
df_filtered['Relay7_Leg_Name']= df_filtered['RelayLeg7_Last_name'] + ', '+ df_filtered['RelayLeg7_First_name']
df_filtered['Relay8_Leg_Name']= df_filtered['RelayLeg8_Last_name'] + ', '+ df_filtered['RelayLeg8_First_name']

df_filtered.drop(['RelayLeg1_Last_name', 'RelayLeg1_First_name', 'RelayLeg2_Last_name', 'RelayLeg2_First_name',
                 'RelayLeg3_Last_name', 'RelayLeg3_First_name', 'RelayLeg4_Last_name', 'RelayLeg4_First_name',
                 'RelayLeg5_Last_name', 'RelayLeg5_First_name', 'RelayLeg6_Last_name', 'RelayLeg6_First_name',
                 'RelayLeg7_Last_name', 'RelayLeg7_First_name', 'RelayLeg8_Last_name', 'RelayLeg8_First_name'], axis=1, inplace=True)


In [58]:
df_filtered.reset_index(inplace = True, drop = True)  # reset index after dropping duplicates


In [59]:
for i in range(len(df_filtered)):
    
    list_no=[]
    list_names=[]
    
    if 'Relay' in df_filtered.loc[i, 'Full_Eventname']:
                
        list_no.append(df_filtered.loc[i, 'RelayLeg1_Ath_no'])
        list_no.append(df_filtered.loc[i, 'RelayLeg2_Ath_no'])
        list_no.append(df_filtered.loc[i, 'RelayLeg3_Ath_no'])
        list_no.append(df_filtered.loc[i, 'RelayLeg4_Ath_no'])
        list_no.append(df_filtered.loc[i, 'RelayLeg5_Ath_no'])
        list_no.append(df_filtered.loc[i, 'RelayLeg6_Ath_no'])
        list_no.append(df_filtered.loc[i, 'RelayLeg7_Ath_no'])
        list_no.append(df_filtered.loc[i, 'RelayLeg8_Ath_no'])
        
        
        list_names.append(df_filtered.loc[i, 'Relay1_Leg_Name'])
        list_names.append(df_filtered.loc[i, 'Relay2_Leg_Name'])
        list_names.append(df_filtered.loc[i, 'Relay3_Leg_Name'])
        list_names.append(df_filtered.loc[i, 'Relay4_Leg_Name'])
        list_names.append(df_filtered.loc[i, 'Relay5_Leg_Name'])
        list_names.append(df_filtered.loc[i, 'Relay6_Leg_Name'])
        list_names.append(df_filtered.loc[i, 'Relay7_Leg_Name'])
        list_names.append(df_filtered.loc[i, 'Relay8_Leg_Name'])
        
        
        list_no = [i for i in list_no if i != 0]
        
        list_names = [i for i in list_names if i != ' ,  ']   
                
        res = {}
        for key in list_no:
            for value in list_names:
                res[key] = value
                list_names.remove(value)
                
                break
        
        df_filtered.at[i, 'Name'] = res  # must use .at for dict
                           
                

In [60]:
df_filtered.drop(['Relay_Ltr', 'RelayLeg1_Ath_no', 'RelayLeg2_Ath_no', 'RelayLeg3_Ath_no', 'RelayLeg4_Ath_no', 
                  'RelayLeg5_Ath_no', 'RelayLeg6_Ath_no', 'RelayLeg7_Ath_no', 'RelayLeg8_Ath_no', 'Relay1_Leg_Name',
                  'Relay2_Leg_Name', 'Relay3_Leg_Name', 'Relay4_Leg_Name', 'Relay5_Leg_Name', 'Relay6_Leg_Name',
                  'Relay7_Leg_Name', 'Relay8_Leg_Name'], axis=1, inplace=True)


In [61]:
df_filtered['Name']=df_filtered['Name'].astype(str) # convert dict to str so duplicates can be dropped


In [62]:
# Extract year

df_filtered['Meet_start'] = pd.to_datetime(df_filtered['Meet_start'])

df_filtered['YEAR'] = df_filtered['Meet_start'].dt.year

df_filtered.drop('Meet_start', axis=1, inplace=True)

In [63]:
df_filtered.columns = ['GENDER', 'DOB', 'AGE', 'TAG_ID', 'TEAM', 'EVENT', 'SEED', 'STAGE', 'HEAT', 'LANE', 'RESULT',
                      'WIND', 'RANK', 'REMARKS', 'ATHLETE_ID', 'DIVISION', 'COMPETITION', 'NAME', 'YEAR']

df_filtered['QUALIFICATION']=''
df_filtered['POINTS']=''
df_filtered['UNIQUE_ID']=''
df_filtered['COUNTRY']=''
df_filtered['DICT_RESULTS']=''
#df_filtered['COMPETITION']='84th Singapore Open'
df_filtered['REGION']='Local'
df_filtered['GROUP']=''
df_filtered['CATEGORY_EVENT']=''
df_filtered['SOURCE']='Meet Manager'
df_filtered['TIMESTAMP']=''
df_filtered['VENUE']=''     # NEW
df_filtered['DATE']=''      # NEW
df_filtered['SESSION']=''   # NEW

# Map miscellaneous fields to final formats                            
                            
mask = df_filtered['STAGE'].str.contains(r'F', na=True)
df_filtered.loc[mask, 'STAGE'] = 'Final'

mask = df_filtered['STAGE'].str.contains(r'P', na=True)
df_filtered.loc[mask, 'STAGE'] = 'Preliminaries'
                                    
mask = df_filtered['GENDER'].isin(['F'])
df_filtered.loc[mask, 'GENDER'] = 'Female'

mask = df_filtered['GENDER'].isin(['M'])
df_filtered.loc[mask, 'GENDER'] = 'Male'

# Map general event category 

df_filtered.loc[df_filtered['EVENT'].str.contains(r'Vault', na=True, case=False), 'CATEGORY_EVENT'] = 'Jump'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Throw', na=True, case=False), 'CATEGORY_EVENT'] = 'Throw'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'100', na=True, case=False), 'CATEGORY_EVENT'] = 'Sprint'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Javelin', na=True, case=False), 'CATEGORY_EVENT'] = 'Throw'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Shot', na=True, case=False), 'CATEGORY_EVENT'] = 'Throw'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'400', na=True, case=False), 'CATEGORY_EVENT'] = 'Sprint'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'600', na=True, case=False), 'CATEGORY_EVENT'] = 'Mid'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'200', na=True, case=False), 'CATEGORY_EVENT'] = 'Sprint'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'50', na=True, case=False), 'CATEGORY_EVENT'] = 'Sprint'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'80', na=True, case=False), 'CATEGORY_EVENT'] = 'Sprint'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'300', na=True, case=False), 'CATEGORY_EVENT'] = 'Sprint'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'3000', na=True, case=False), 'CATEGORY_EVENT'] = 'Long'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Mile', na=True, case=False), 'CATEGORY_EVENT'] = 'Mid'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'5000', na=True, case=False), 'CATEGORY_EVENT'] = 'Long'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'2000', na=True, case=False), 'CATEGORY_EVENT'] = 'Mid'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'800', na=True, case=False), 'CATEGORY_EVENT'] = 'Mid'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'1500', na=True, case=False), 'CATEGORY_EVENT'] = 'Mid'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Discus', na=True, case=False), 'CATEGORY_EVENT'] = 'Throw'

# Override values

df_filtered.loc[df_filtered['EVENT'].str.contains(r'Jump', na=True, case=False), 'CATEGORY_EVENT'] = 'Jump'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Relay', na=True, case=False), 'CATEGORY_EVENT'] = 'Relay'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Steeple', na=True, case=False), 'CATEGORY_EVENT'] = 'Steeple'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Walk', na=True, case=False), 'CATEGORY_EVENT'] = 'Walk'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Hurdles', na=True, case=False), 'CATEGORY_EVENT'] = 'Hurdles'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Pentathlon', na=True, case=False), 'CATEGORY_EVENT'] = 'Pentathlon'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Triathlon', na=True, case=False), 'CATEGORY_EVENT'] = 'Triathlon'


# Strip extraneous values from DOB

df_filtered['DOB']=df_filtered.DOB.str.rstrip('00:00:00.000') 
    

# Rearrange columns    
    
df_filtered = df_filtered.reindex(columns= ['RANK', 'TAG_ID', 'NAME', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION', 'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 
                                      'POINTS', 'AGE', 'GENDER', 'UNIQUE_ID', 'COUNTRY', 'DICT_RESULTS', 'YEAR', 'COMPETITION', 'REGION', 'DOB','GROUP', 'CATEGORY_EVENT', 
                                      'ATHLETE_ID', 'SOURCE', 'REMARKS', 'TIMESTAMP', 'VENUE', 'DATE', 'SESSION'])


In [64]:
# NEW ADDITIONS

df_filtered.loc[df_filtered['EVENT'].str.contains(r'Petathlon', na=True, case=False), 'SUB_EVENT'] = 'Pentathlon'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Triathlon', na=True, case=False), 'SUB_EVENT'] = 'Triathlon'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Heptathlon', na=True, case=False), 'SUB_EVENT'] = 'Heptathlon'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Mixed', na=True, case=False), 'SUB_EVENT'] = 'Mixed'


df_filtered.loc[df_filtered['EVENT'].str.contains(r'Long Jump', na=True, case=False), 'EVENT'] = 'Long jump'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Triple Jump', na=True, case=False), 'EVENT'] = 'Triple jump'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Javelin Throw', na=True, case=False), 'EVENT'] = 'Javeline throw'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'60.0 Dash', na=True, case=False), 'EVENT'] = '60m'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'80.0 Dash', na=True, case=False), 'EVENT'] = '80m'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'80 Meter', na=True, case=False), 'EVENT'] = '80m'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'100 Meter Dash', na=True, case=False), 'EVENT'] = '100m'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'200 Meter Dash', na=True, case=False), 'EVENT'] = '100m'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'400 Meter Dash', na=True, case=False), 'EVENT'] = '400m'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'800.0 Run', na=True, case=False), 'EVENT'] = '800m'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'100.0 Hurdles', na=True, case=False), 'EVENT'] = '100m hurdles'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'1500 Meter Race Walk', na=True, case=False), 'EVENT'] = '1500m race walk'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'1500 Meter Run', na=True, case=False), 'EVENT'] = '1500m run'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'4x100 Meter Relay', na=True, case=False), 'EVENT'] = '4x100m relay'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'400 Meter Hurdles', na=True, case=False), 'EVENT'] = '400m hurdles'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'4x50 Meter Relay', na=True, case=False), 'EVENT'] = '4x50m relay'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'4x1000 Meter Relay', na=True, case=False), 'EVENT'] = '4x1000m relay'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Discus Throw', na=True, case=False), 'EVENT'] = 'Discus throw'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'110 Meter Hurdles', na=True, case=False), 'EVENT'] = '110m hurdles'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'100 Meter Hurdles', na=True, case=False), 'EVENT'] = '100m hurdles'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'80.0 Hurdles', na=True, case=False), 'EVENT'] = '80m hurdles'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'800.0 Run', na=True, case=False), 'EVENT'] = '800m'


In [65]:
df_filtered['EVENT']

0                          100m
1                          100m
2                          100m
3                          100m
4                          100m
                 ...           
290                   Long jump
291                   Long jump
292    Men 16-99 High Jump Open
293    Men 16-99 High Jump Open
294    Men 16-99 High Jump Open
Name: EVENT, Length: 295, dtype: object

In [66]:
df_filtered['CATEGORY_EVENT']

0      Sprint
1      Sprint
2      Sprint
3      Sprint
4      Sprint
        ...  
290      Jump
291      Jump
292      Jump
293      Jump
294      Jump
Name: CATEGORY_EVENT, Length: 295, dtype: object

In [67]:
# NEW ADDITIONS

assert (~(df_filtered.loc[df_filtered['EVENT'].str.contains(r'Triathlon', na=True, case=False), 'CATEGORY_EVENT'] == 'Triathlon')).values.sum()==0
assert (~(df_filtered.loc[df_filtered['EVENT'].str.contains(r'Pentathlon', na=True, case=False), 'CATEGORY_EVENT'] == 'Pentathlon')).values.sum()==0
assert (~(df_filtered.loc[df_filtered['EVENT'].str.contains(r'Pentathlon', na=True, case=False), 'CATEGORY_EVENT'] == 'Pentathlon')).values.sum()==0
assert (~(df_filtered.loc[df_filtered['EVENT'].str.contains(r'Walk', na=True, case=False), 'CATEGORY_EVENT'] == 'Walk')).values.sum()==0
assert (~(df_filtered.loc[df_filtered['EVENT'].str.contains(r'Steeple', na=True, case=False), 'CATEGORY_EVENT'] == 'Steeple')).values.sum()==0
assert (~(df_filtered.loc[df_filtered['EVENT'].str.contains(r'Relay', na=True, case=False), 'CATEGORY_EVENT'] == 'Relay')).values.sum()==0
#assert (~(df_filtered.loc[df_filtered['EVENT'].str.contains(r'^(?!.*Triathlon|Pentathlon|Heptathlon).*Jump.*$', na=True, case=False), 'CATEGORY_EVENT'] == 'Jump')).values.sum()==0
assert (~(df_filtered.loc[df_filtered['EVENT'].str.contains(r'Discus', na=True, case=False), 'CATEGORY_EVENT'] == 'Throw')).values.sum()==0
assert (~(df_filtered.loc[df_filtered['EVENT'].str.contains(r'Shot', na=True, case=False), 'CATEGORY_EVENT'] == 'Throw')).values.sum()==0
assert (~(df_filtered.loc[df_filtered['EVENT'].str.contains(r'Javelin', na=True, case=False), 'CATEGORY_EVENT'] == 'Throw')).values.sum()==0
assert (~(df_filtered.loc[df_filtered['EVENT'].str.contains(r'/^.*?\b100m\b.*?\bhurdles\b.*?$/m', na=True, case=False), 'CATEGORY_EVENT'] == 'Hurdles')).values.sum()==0
assert (~(df_filtered.loc[df_filtered['EVENT'].str.contains(r'/^.*?\b400m\b.*?\bhurdles\b.*?$/m', na=True, case=False), 'CATEGORY_EVENT'] == 'Hurdles')).values.sum()==0



In [68]:
(~(df_filtered.loc[df_filtered['EVENT'].str.contains(r'^(?!.*Pentathlon).*Hurdles.*$', na=True, case=False), 'CATEGORY_EVENT'] == 'Hurdles'))


186    False
187    False
188    False
189    False
190    False
191    False
192    False
193    False
194    False
195    False
196    False
197    False
198    False
199    False
200    False
201    False
202    False
203    False
204    False
205    False
206    False
207    False
Name: CATEGORY_EVENT, dtype: bool

In [69]:
df_filtered.loc[3, 'EVENT']

'100m'

In [70]:
df_filtered.to_csv('Processed -GenericData-SSP Invitational U15 and U18 2025-31May2025-001.csv', sep=',', encoding='utf-8-sig', index=False)

## Check for errors in reporting DOB and Gender in MM

In [1904]:
###
# Check for inconsistencies in DOB and Gender
###

# LATEST VERSION
# Reading and processing CSV output from MM DB -- CTE results combining meet info
# Use SQL Query 
#WITH test AS (
#          	SELECT * FROM [Results]
#			LEFT JOIN [Athlete] ON [Athlete].[Comp_no] = [Results].[Comp_no]
#			LEFT JOIN [Entries] ON [Entries].[Comp_no] = [Results].[Comp_no] AND [Entries].[Event_numltr]=[Results].[Event_numltr] 
#			JOIN [Divisions] ON [Divisions].[Div_no]  = [Results].[Div_no]
#			JOIN [Team] ON [Results].[Team_no] = [Team].[Team_no]
#
#        )
#        SELECT * FROM test, [Meet]

# Comp_no = TAG ID

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/mm_files/2024/')

df = pd.read_csv("GenericData-SMTFA Track - Field Championship 2024-22Jun2024-001.csv")


In [1905]:
list(df.columns)

['Event_numltr',
 'Rnd_ltr',
 'MultiSubEvent_no',
 'Trk_Field',
 'Ind_rel',
 'Event_gender',
 'Event_dist',
 'Event_code',
 'Event_name',
 'Full_Eventname',
 'MultiSubEvent_name',
 'low_age',
 'high_age',
 'Div_no',
 'Event_note',
 'Event_Type',
 'Entry_fee',
 'Ath_no',
 'Comp_no',
 'Last_name',
 'First_name',
 'Reg_no',
 'Team_no',
 'Team_Abbr',
 'Team_lsc',
 'Relay_Ltr',
 'Event_score',
 'Res_heat',
 'Res_lane',
 'Res_stat',
 'Res_markDisplay',
 'Res_mark',
 'Res_meas',
 'Res_handtime',
 'Res_wind',
 'Res_agegradeDisplay',
 'Res_agegrade',
 'Res_percent',
 'Res_heatplace',
 'Res_place',
 'Res_exh',
 'Res_note',
 'RelayLeg1_Ath_no',
 'RelayLeg1_Last_name',
 'RelayLeg1_First_name',
 'RelayLeg2_Ath_no',
 'RelayLeg2_Last_name',
 'RelayLeg2_First_name',
 'RelayLeg3_Ath_no',
 'RelayLeg3_Last_name',
 'RelayLeg3_First_name',
 'RelayLeg4_Ath_no',
 'RelayLeg4_Last_name',
 'RelayLeg4_First_name',
 'RelayLeg5_Ath_no',
 'RelayLeg5_Last_name',
 'RelayLeg5_First_name',
 'RelayLeg6_Ath_no',
 'RelayL

In [1906]:
df_check = df[['Last_name', 'First_name', 'Event_gender', 'Birth_date', 'Ath_age', 'Ath_Sex', 'Comp_no', 'Ath_no','Event_name', 'Full_Eventname', 'Full_Eventname.1', 'Event_gender.1', 'Meet_name']].copy()


In [1907]:
df_check

,Last_name,First_name,Event_gender,Birth_date,Ath_age,Ath_Sex,Comp_no,Ath_no,Event_name,Full_Eventname,Full_Eventname.1,Event_gender.1,Meet_name
0,Saha,Sayantani,F,1981-06-06 00:00:00.000,43.0,F,4111,278,Run,W40 1500 Meter Run Masters,W40 1500 Meter Run Masters,F,SMTFA Track & Field Championship 2024
1,Wei Ping,Lim,F,1978-04-14 00:00:00.000,46.0,F,4603,312,Run,W45 1500 Meter Run Masters,W45 1500 Meter Run Masters,F,SMTFA Track & Field Championship 2024
2,Su Yin,Ong,F,1978-10-25 00:00:00.000,45.0,F,4605,313,Run,W45 1500 Meter Run Masters,W45 1500 Meter Run Masters,F,SMTFA Track & Field Championship 2024
3,Haniza Razak,RazLiza,F,1977-03-04 00:00:00.000,47.0,F,4606,316,Run,W45 1500 Meter Run Masters,W45 1500 Meter Run Masters,F,SMTFA Track & Field Championship 2024
4,Cheong,Michelle,F,1965-12-11 00:00:00.000,58.0,F,5609,430,Run,W55 1500 Meter Run Masters,W55 1500 Meter Run Masters,F,SMTFA Track & Field Championship 2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2621,Fernando,Hapuhannedige Sisura,M,2001-02-13 00:00:00.000,23.0,M,2505,212,Long Jump,Men Long Jump Open,Men Long Jump Open,M,SMTFA Track & Field Championship 2024
2622,Bliey,Jackson,M,2003-07-10 00:00:00.000,20.0,M,2518,804,Javelin Throw,Men Javelin Throw Open,Men Javelin Throw Open,M,SMTFA Track & Field Championship 2024
2623,Meena,Girish Chandramohan,M,1995-11-20 00:00:00.000,28.0,M,2504,821,Javelin Throw,Men Javelin Throw Open,Men Javelin Throw Open,M,SMTFA Track & Field Championship 2024
2624,Tissera,Ashen,M,2002-07-16 00:00:00.000,21.0,M,2501,205,Shot Put,Men Shot Put Open,Men Shot Put Open,M,SMTFA Track & Field Championship 2024


In [1908]:
df_check['Event_gender'].equals(df_check['Ath_Sex'])

False

In [1909]:
df_check['Event_gender'].equals(df_check['Event_gender.1'])

True

In [1910]:
df_check['all_matching']=df_check.apply(lambda x: x['Event_gender'] == x['Event_gender.1'] == x['Ath_Sex'], axis = 1)

In [1911]:
df_check

,Last_name,First_name,Event_gender,Birth_date,Ath_age,Ath_Sex,Comp_no,Ath_no,Event_name,Full_Eventname,Full_Eventname.1,Event_gender.1,Meet_name,all_matching
0,Saha,Sayantani,F,1981-06-06 00:00:00.000,43.0,F,4111,278,Run,W40 1500 Meter Run Masters,W40 1500 Meter Run Masters,F,SMTFA Track & Field Championship 2024,True
1,Wei Ping,Lim,F,1978-04-14 00:00:00.000,46.0,F,4603,312,Run,W45 1500 Meter Run Masters,W45 1500 Meter Run Masters,F,SMTFA Track & Field Championship 2024,True
2,Su Yin,Ong,F,1978-10-25 00:00:00.000,45.0,F,4605,313,Run,W45 1500 Meter Run Masters,W45 1500 Meter Run Masters,F,SMTFA Track & Field Championship 2024,True
3,Haniza Razak,RazLiza,F,1977-03-04 00:00:00.000,47.0,F,4606,316,Run,W45 1500 Meter Run Masters,W45 1500 Meter Run Masters,F,SMTFA Track & Field Championship 2024,True
4,Cheong,Michelle,F,1965-12-11 00:00:00.000,58.0,F,5609,430,Run,W55 1500 Meter Run Masters,W55 1500 Meter Run Masters,F,SMTFA Track & Field Championship 2024,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2621,Fernando,Hapuhannedige Sisura,M,2001-02-13 00:00:00.000,23.0,M,2505,212,Long Jump,Men Long Jump Open,Men Long Jump Open,M,SMTFA Track & Field Championship 2024,True
2622,Bliey,Jackson,M,2003-07-10 00:00:00.000,20.0,M,2518,804,Javelin Throw,Men Javelin Throw Open,Men Javelin Throw Open,M,SMTFA Track & Field Championship 2024,True
2623,Meena,Girish Chandramohan,M,1995-11-20 00:00:00.000,28.0,M,2504,821,Javelin Throw,Men Javelin Throw Open,Men Javelin Throw Open,M,SMTFA Track & Field Championship 2024,True
2624,Tissera,Ashen,M,2002-07-16 00:00:00.000,21.0,M,2501,205,Shot Put,Men Shot Put Open,Men Shot Put Open,M,SMTFA Track & Field Championship 2024,True


In [1912]:
df_check.to_csv('gender_check.csv', sep=',', encoding='utf-8-sig', index=False)

## Process CSV Results Table Missing from MDB File

In [129]:
# Reading and processing MM results csv

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/CSV_results/')

results = pd.read_csv("2023_allcomers_2.csv", encoding='windows-1252', header=None)  # do not read first row as column names


In [130]:
results

,0,1,2,3,4,5,6,7,8,9,...,185,186,187,188,189,190,191,192,193,194
0,Singapore Athletics Assoc.-Open Champ - Organi...,Hy-Tek's MEET MANAGER 1:31 PM 20/9/2024 Page 1,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,NaN,Results,,Event 101 Boys Long Jump A Div,Finals ... (Event 101 Boys Long Jump A Div),NaN,...,NaN,NaN,,,0,NaN,NaN,NaN,NaN,NaN
1,Singapore Athletics Assoc.-Open Champ - Organi...,Hy-Tek's MEET MANAGER 1:31 PM 20/9/2024 Page 1,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,NaN,Results,,Event 101 Boys Long Jump A Div,Finals ... (Event 101 Boys Long Jump A Div),NaN,...,NaN,NaN,,,0,NaN,NaN,NaN,NaN,NaN
2,Singapore Athletics Assoc.-Open Champ - Organi...,Hy-Tek's MEET MANAGER 1:31 PM 20/9/2024 Page 1,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,NaN,Results,,Event 101 Boys Long Jump A Div,Finals ... (Event 101 Boys Long Jump A Div),NaN,...,NaN,NaN,,,0,NaN,NaN,NaN,NaN,NaN
3,Singapore Athletics Assoc.-Open Champ - Organi...,Hy-Tek's MEET MANAGER 1:31 PM 20/9/2024 Page 1,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,NaN,Results,,Event 101 Boys Long Jump A Div,Finals ... (Event 101 Boys Long Jump A Div),NaN,...,NaN,NaN,,,0,NaN,NaN,NaN,NaN,NaN
4,Singapore Athletics Assoc.-Open Champ - Organi...,Hy-Tek's MEET MANAGER 1:31 PM 20/9/2024 Page 1,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,NaN,Results,,Event 101 Boys Long Jump A Div,Finals ... (Event 101 Boys Long Jump A Div),NaN,...,NaN,NaN,,,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2123,Singapore Athletics Assoc.-Open Champ - Organi...,Hy-Tek's MEET MANAGER 1:31 PM 20/9/2024 Page 1,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,NaN,Results,,Event 447 Men 800 Meter Run Open,Finals ... (Event 447 Men 800 Meter Run Open),NaN,...,NaN,NaN,1,,0,NaN,NaN,NaN,NaN,NaN
2124,Singapore Athletics Assoc.-Open Champ - Organi...,Hy-Tek's MEET MANAGER 1:31 PM 20/9/2024 Page 1,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,NaN,Results,,Event 447 Men 800 Meter Run Open,Finals ... (Event 447 Men 800 Meter Run Open),NaN,...,NaN,NaN,3,,0,NaN,NaN,NaN,NaN,NaN
2125,Singapore Athletics Assoc.-Open Champ - Organi...,Hy-Tek's MEET MANAGER 1:31 PM 20/9/2024 Page 1,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,NaN,Results,,Event 447 Men 800 Meter Run Open,Finals ... (Event 447 Men 800 Meter Run Open),NaN,...,NaN,NaN,2,,0,NaN,NaN,NaN,NaN,NaN
2126,Singapore Athletics Assoc.-Open Champ - Organi...,Hy-Tek's MEET MANAGER 1:31 PM 20/9/2024 Page 1,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,NaN,Results,,Event 712 Girls Heptathlon U18,Finals ... (Event 712 Girls Heptathlon U18),NaN,...,NaN,NaN,,,0,NaN,NaN,NaN,NaN,NaN


In [131]:
results_dropped = results.T.drop_duplicates().T

In [132]:
results_dropped

,0,1,2,3,4,5,6,7,8,64,...,91,92,94,95,96,104,191,192,193,194
0,Singapore Athletics Assoc.-Open Champ - Organi...,Hy-Tek's MEET MANAGER 1:31 PM 20/9/2024 Page 1,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,NaN,Results,,Event 101 Boys Long Jump A Div,Finals ... (Event 101 Boys Long Jump A Div),Name,...,VICTORIA JUNIOR COLLEGE,6.10m,,,0,0,NaN,NaN,NaN,NaN
1,Singapore Athletics Assoc.-Open Champ - Organi...,Hy-Tek's MEET MANAGER 1:31 PM 20/9/2024 Page 1,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,NaN,Results,,Event 101 Boys Long Jump A Div,Finals ... (Event 101 Boys Long Jump A Div),Name,...,RAFFLES INSTITUTION JC,6.01m,,,0,0,NaN,NaN,NaN,NaN
2,Singapore Athletics Assoc.-Open Champ - Organi...,Hy-Tek's MEET MANAGER 1:31 PM 20/9/2024 Page 1,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,NaN,Results,,Event 101 Boys Long Jump A Div,Finals ... (Event 101 Boys Long Jump A Div),Name,...,SINGAPORE SPORTS SCHOOL,5.63m,,,0,0,NaN,NaN,NaN,NaN
3,Singapore Athletics Assoc.-Open Champ - Organi...,Hy-Tek's MEET MANAGER 1:31 PM 20/9/2024 Page 1,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,NaN,Results,,Event 101 Boys Long Jump A Div,Finals ... (Event 101 Boys Long Jump A Div),Name,...,RAFFLES INSTITUTION JC,5.34m,,,0,-0.2,NaN,NaN,NaN,NaN
4,Singapore Athletics Assoc.-Open Champ - Organi...,Hy-Tek's MEET MANAGER 1:31 PM 20/9/2024 Page 1,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,NaN,Results,,Event 101 Boys Long Jump A Div,Finals ... (Event 101 Boys Long Jump A Div),Name,...,RAFFLES INSTITUTION JC,5.18m,,,0,-0.8,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2123,Singapore Athletics Assoc.-Open Champ - Organi...,Hy-Tek's MEET MANAGER 1:31 PM 20/9/2024 Page 1,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,NaN,Results,,Event 447 Men 800 Meter Run Open,Finals ... (Event 447 Men 800 Meter Run Open),Name,...,LACTICBUDS,DNS,1,,0,,NaN,NaN,NaN,NaN
2124,Singapore Athletics Assoc.-Open Champ - Organi...,Hy-Tek's MEET MANAGER 1:31 PM 20/9/2024 Page 1,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,NaN,Results,,Event 447 Men 800 Meter Run Open,Finals ... (Event 447 Men 800 Meter Run Open),Name,...,WINGS ATHLETICS CLUB,DNS,3,,0,,NaN,NaN,NaN,NaN
2125,Singapore Athletics Assoc.-Open Champ - Organi...,Hy-Tek's MEET MANAGER 1:31 PM 20/9/2024 Page 1,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,NaN,Results,,Event 447 Men 800 Meter Run Open,Finals ... (Event 447 Men 800 Meter Run Open),Name,...,OLDHAM ATHLETICS,DNS,2,,0,,NaN,NaN,NaN,NaN
2126,Singapore Athletics Assoc.-Open Champ - Organi...,Hy-Tek's MEET MANAGER 1:31 PM 20/9/2024 Page 1,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,NaN,Results,,Event 712 Girls Heptathlon U18,Finals ... (Event 712 Girls Heptathlon U18),Name,...,SINGAPORE SPORTS SCHOOL,2955,,,0,,NaN,NaN,NaN,NaN


In [133]:
results_cleaned = results_dropped.drop(results_dropped.iloc[:, 8:14],axis = 1)
results_cleaned=results_cleaned.drop([70, 69, 6, 5, 4, 1], axis=1)

In [134]:
results_cleaned

,0,2,3,7,88,89,90,91,92,94,95,96,104,191,192,193,194
0,Singapore Athletics Assoc.-Open Champ - Organi...,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,Event 101 Boys Long Jump A Div,1,"#1993 Emery, Conrad",17.0,VICTORIA JUNIOR COLLEGE,6.10m,,,0,0,NaN,NaN,NaN,NaN
1,Singapore Athletics Assoc.-Open Champ - Organi...,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,Event 101 Boys Long Jump A Div,2,"#1601 Ho, Joshua",18.0,RAFFLES INSTITUTION JC,6.01m,,,0,0,NaN,NaN,NaN,NaN
2,Singapore Athletics Assoc.-Open Champ - Organi...,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,Event 101 Boys Long Jump A Div,3,"#1743 Zainal, Ezra Nur Iman",18.0,SINGAPORE SPORTS SCHOOL,5.63m,,,0,0,NaN,NaN,NaN,NaN
3,Singapore Athletics Assoc.-Open Champ - Organi...,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,Event 101 Boys Long Jump A Div,4,"#1603 Hossain, Leo",18.0,RAFFLES INSTITUTION JC,5.34m,,,0,-0.2,NaN,NaN,NaN,NaN
4,Singapore Athletics Assoc.-Open Champ - Organi...,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,Event 101 Boys Long Jump A Div,5,"#1618 Ng, Tze Yuan",18.0,RAFFLES INSTITUTION JC,5.18m,,,0,-0.8,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2123,Singapore Athletics Assoc.-Open Champ - Organi...,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,Event 447 Men 800 Meter Run Open,---,"#1163 Muhammad, Faizal",22.0,LACTICBUDS,DNS,1,,0,,NaN,NaN,NaN,NaN
2124,Singapore Athletics Assoc.-Open Champ - Organi...,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,Event 447 Men 800 Meter Run Open,---,"#2142 Quek, Calvin",27.0,WINGS ATHLETICS CLUB,DNS,3,,0,,NaN,NaN,NaN,NaN
2125,Singapore Athletics Assoc.-Open Champ - Organi...,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,Event 447 Men 800 Meter Run Open,---,"#1424 Soh, Wee jin",15.0,OLDHAM ATHLETICS,DNS,2,,0,,NaN,NaN,NaN,NaN
2126,Singapore Athletics Assoc.-Open Champ - Organi...,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,Event 712 Girls Heptathlon U18,1,"#1712 Syed Ahmed Riadh, Sharifah Falisha",17.0,SINGAPORE SPORTS SCHOOL,2955,,,0,,NaN,NaN,NaN,NaN


In [135]:
df=results_cleaned

df=df.astype(str)

In [136]:
df[7]

0         Event 101  Boys Long Jump A Div
1         Event 101  Boys Long Jump A Div
2         Event 101  Boys Long Jump A Div
3         Event 101  Boys Long Jump A Div
4         Event 101  Boys Long Jump A Div
                      ...                
2123    Event 447  Men 800 Meter Run Open
2124    Event 447  Men 800 Meter Run Open
2125    Event 447  Men 800 Meter Run Open
2126      Event 712  Girls Heptathlon U18
2127      Event 712  Girls Heptathlon U18
Name: 7, Length: 2128, dtype: object

In [137]:
# Apply regex to extract info column wise

#df['XXX'] =  df[7].apply(lambda x: re.search(r'Event\s\d\d\d\s', str(x)))


In [203]:
# Apply a function to extract data using regex

def event(column):
    
    spos=re.search(r'(Boys|Girls|Women|Men)', column)
    epos=re.search(r'(A Div|B Div|C Div|D Div|Para|Open|Novice|U18|Intermediate|Advance|Trial)', column)
        
    split_start=spos.end()+1
    split_end=epos.start()-1
    event=column[split_start:split_end]
    
    print(event)
    
    return event

def athno(column):
    
    e=re.findall(r'(\d\d\d\d|\d\d\d)', column)
    
    return e

def sex(column):
    
    e=re.findall(r'(Boys|Girls|Men|Women)', column)
    
    return e

def firstname(column):
    
    pos=re.search(r'(\d\d\d\d|\d\d\d)', column)
 #   sin=re.search(r'SINGAPORE', column)
    
    if 'SINGAPORE' not in column:  # exclude relay events that have no names in this column
        split_start=pos.end()+1
        name=column[split_start:]
    else:
        name=''
    return name

def meet(column):
    
    pos=re.search(r'\s\-\s', column)
    
    split_end=pos.start()
    
    meet=column[:split_end]
    
    return meet


In [187]:
df['Full_eventname']=df[7].apply(event)

Long Jump
Long Jump
Long Jump
Long Jump
Long Jump
Long Jump
Long Jump
Long Jump
Long Jump
Long Jump
Javelin Throw (600g)
Javelin Throw (600g)
Javelin Throw (600g)
Javelin Throw (600g)
Javelin Throw (600g)
Javelin Throw (600g)
Javelin Throw (500g)
Javelin Throw (500g)
Javelin Throw (500g)
Javelin Throw (500g)
Javelin Throw (500g)
Javelin Throw (500g)
Javelin Throw (500g)
Javelin Throw (500g)
Javelin Throw (500g)
5000 Meter Race Walk
5000 Meter Race Walk
5000 Meter Race Walk
5000 Meter Race Walk
5000 Meter Race Walk
5000 Meter Race Walk
5000 Meter Race Walk
Long Jump
Long Jump
Long Jump
Long Jump
Long Jump
Long Jump
Long Jump
Long Jump
Long Jump
Long Jump
Long Jump
Long Jump
10000 Meter Run
10000 Meter Run
10000 Meter Run
10000 Meter Run
10000 Meter Run
10000 Meter Run
Shot Put (5kg)
Shot Put (5kg)
Shot Put (5kg)
Shot Put (5kg)
Shot Put (5kg)
Shot Put (7.26kg)
Long Jump
Long Jump
Long Jump
Long Jump
Long Jump
Long Jump
Long Jump
Long Jump
Long Jump
Long Jump
Long Jump
Long Jump
Long Jump

In [184]:
df['Ath_no']=df[89].apply(athno)

In [185]:
df['sex']=df[7].apply(sex)

In [198]:
df['First_name']=df[89].apply(firstname)

In [204]:
df['meet']=df[2].apply(meet)

In [205]:
df

,0,2,3,7,88,89,90,91,92,94,...,191,192,193,194,check1,Ath_no,sex,Full_eventname,First_name,meet
0,Singapore Athletics Assoc.-Open Champ - Organi...,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,Event 101 Boys Long Jump A Div,1,"#1993 Emery, Conrad",17.0,VICTORIA JUNIOR COLLEGE,6.10m,,...,nan,nan,nan,nan,Long Jump,[1993],[Boys],Long Jump,"Emery, Conrad",SA Allcomers Meet 2
1,Singapore Athletics Assoc.-Open Champ - Organi...,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,Event 101 Boys Long Jump A Div,2,"#1601 Ho, Joshua",18.0,RAFFLES INSTITUTION JC,6.01m,,...,nan,nan,nan,nan,Long Jump,[1601],[Boys],Long Jump,"Ho, Joshua",SA Allcomers Meet 2
2,Singapore Athletics Assoc.-Open Champ - Organi...,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,Event 101 Boys Long Jump A Div,3,"#1743 Zainal, Ezra Nur Iman",18.0,SINGAPORE SPORTS SCHOOL,5.63m,,...,nan,nan,nan,nan,Long Jump,[1743],[Boys],Long Jump,"Zainal, Ezra Nur Iman",SA Allcomers Meet 2
3,Singapore Athletics Assoc.-Open Champ - Organi...,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,Event 101 Boys Long Jump A Div,4,"#1603 Hossain, Leo",18.0,RAFFLES INSTITUTION JC,5.34m,,...,nan,nan,nan,nan,Long Jump,[1603],[Boys],Long Jump,"Hossain, Leo",SA Allcomers Meet 2
4,Singapore Athletics Assoc.-Open Champ - Organi...,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,Event 101 Boys Long Jump A Div,5,"#1618 Ng, Tze Yuan",18.0,RAFFLES INSTITUTION JC,5.18m,,...,nan,nan,nan,nan,Long Jump,[1618],[Boys],Long Jump,"Ng, Tze Yuan",SA Allcomers Meet 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2123,Singapore Athletics Assoc.-Open Champ - Organi...,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,Event 447 Men 800 Meter Run Open,---,"#1163 Muhammad, Faizal",22.0,LACTICBUDS,DNS,1,...,nan,nan,nan,nan,800 Meter Run,[1163],[Men],800 Meter Run,"Muhammad, Faizal",SA Allcomers Meet 2
2124,Singapore Athletics Assoc.-Open Champ - Organi...,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,Event 447 Men 800 Meter Run Open,---,"#2142 Quek, Calvin",27.0,WINGS ATHLETICS CLUB,DNS,3,...,nan,nan,nan,nan,800 Meter Run,[2142],[Men],800 Meter Run,"Quek, Calvin",SA Allcomers Meet 2
2125,Singapore Athletics Assoc.-Open Champ - Organi...,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,Event 447 Men 800 Meter Run Open,---,"#1424 Soh, Wee jin",15.0,OLDHAM ATHLETICS,DNS,2,...,nan,nan,nan,nan,800 Meter Run,[1424],[Men],800 Meter Run,"Soh, Wee jin",SA Allcomers Meet 2
2126,Singapore Athletics Assoc.-Open Champ - Organi...,SA Allcomers Meet 2 - 3/2/2023 to 11/2/2023,Home of Athletics,Event 712 Girls Heptathlon U18,1,"#1712 Syed Ahmed Riadh, Sharifah Falisha",17.0,SINGAPORE SPORTS SCHOOL,2955,,...,nan,nan,nan,nan,Heptathlon,[1712],[Girls],Heptathlon,"Syed Ahmed Riadh, Sharifah Falisha",SA Allcomers Meet 2


In [209]:
results_cleaned.to_csv('results_cleaned.csv', sep=',', encoding='utf-8-sig', index=False)

In [175]:
results_dropped.reindex(columns=['', 'user_agent'])

NameError: name 'column' is not defined

In [ ]:
#df4 = df4.drop(df4.iloc[:, 8:11],axis = 1)

